In [1]:
# Top

This notebook generates the individual files for the dero package. When making changes, be sure to update the version in the top cell. Then restart kernel and run all cells, then navigate to the root folder (outside dero), and run upload.bat. 

TODO

In [2]:
version = '0.8.1'
upload = False

In [3]:
version_info = tuple([int(i) for i in version.split('.')]) #tuple of ints, e.g. (0,2,2)

with open('version.py', 'w') as f:
    f.write('__version__ = "{}" \n'.format(version))
    f.write('__version_info__ = {}'.format(version_info))

In [4]:
%%writefile requirements.txt
pandas
pandasql
wrds
numpy
pdfrw
selenium
unidecode
IPython
sas7bdat
statsmodels
matplotlib
sympy
pandastable

Overwriting requirements.txt


In [5]:
%%writefile setup.py

from version import __version__
from setuptools import setup


setup(name='Dero',
      version=__version__,
      description="Nick DeRobertis Personal Library",
      long_description='''
      Nick DeRobertis' personal library of functions. This is hosted on PyPi mostly for my own 
      convenience, though others may use it so long as I'm given credit. 
      ''',
      author='Nick DeRobertis',
      author_email='whoopnip@gmail.com',
      license='MIT',
      packages=['dero'],
      classifiers=[
        # How mature is this project? Common values are
        #   3 - Alpha
        #   4 - Beta
        #   5 - Production/Stable
        'Development Status :: 3 - Alpha',

        # Indicate who your project is intended for
        'Intended Audience :: Developers',

        # Specify the Python versions you support here. In particular, ensure
        # that you indicate whether you support Python 2, Python 3 or both.
        'Programming Language :: Python :: 3',
        'Programming Language :: Python :: 3.2',
        'Programming Language :: Python :: 3.3',
        'Programming Language :: Python :: 3.4',
        'Programming Language :: Python :: 3.5'
        ],
       install_requires=[
        'pandas',
        'pandasql',
        'numpy',
        'wrds',
        'pdfrw',
        'selenium',
        'unidecode',
        'IPython',
        'sas7bdat',
        'statsmodels',
        'matplotlib',
        'sympy',
        'pandastable'],
     )

Overwriting setup.py


In [6]:
import os

orig_path = os.getcwd()

os.chdir('dero')

# init

In [7]:
init_str = '''
__version__ = "{}"
__version_info__ = {}

#Provided for backwards compatibility with old code. In the old code, add to the top:
#import builtins
#builtins.__dero_version__ = '0.1.0'
try: 
    __dero_version__
    if __dero_version__ == '0.1.0':
        from dero.core import *
        from dero.data import *
        from dero.decorators import *
        from dero.latex import *
        from dero.pdf import *
        
        from dero.ext_logging import *
        from dero.ext_matplotlib import *
        from dero.ext_multiprocessing import *
        from dero.ext_pandas import *
        from dero.ext_selenium import *
        from dero.ext_sympy import *
        from dero.ext_time import *
        pandas_to_csv = to_csv
    else: #stuctured this way so that we can support future structuring changes, can add additional if statements
        __dero_version__ = 'current'
except NameError:
    __dero_version__ = 'current'

if __dero_version__ == 'current':
    import dero.core
    import dero.data
    import dero.decorators
    import dero.latex
    import dero.pdf
    import dero.wrds
    import dero.logtimer
    import dero.reg
    import dero.summ
    
    import dero.ext_logging as logging
    import dero.ext_matplotlib as matplotlib
    import dero.ext_multiprocessing as multiprocessing
    import dero.ext_pandas as pandas
    import dero.ext_selenium as selenium
    import dero.ext_sympy as sympy
    import dero.ext_time as time
    import dero.ext_math as math
    
    del dero.ext_logging
    del dero.ext_matplotlib
    del dero.ext_multiprocessing
    del dero.ext_pandas
    del dero.ext_selenium
    del dero.ext_sympy
    del dero.ext_time
    del dero.ext_math
    
#     sys.modules['dero.logging'] = dero.ext_logging
#     sys.modules['dero.matplotlib'] = dero.ext_matplotlib
#     sys.modules['dero.multiprocessing'] = dero.ext_multiprocessing
#     sys.modules['dero.pandas'] = dero.ext_pandas
#     sys.modules['dero.selenium'] = dero.ext_selenium
#     sys.modules['dero.sympy'] = dero.ext_sympy
#     sys.modules['dero.time'] = dero.ext_time
    
#     import dero.logging
#     import dero.matplotlib
#     import dero.multiprocessing
#     import dero.pandas
#     import dero.selenium
#     import dero.sympy
#     import dero.time
    
     

'''.format(version, version_info)

with open('__init__.py', 'w') as f:
    f.write(init_str)

# Pandas

In [8]:
%%writefile ext_pandas.py

import os
import pandas as pd
import datetime, time
from numpy import nan
from dateutil.relativedelta import relativedelta
import sys
import numpy as np
import warnings, timeit
from sas7bdat import SAS7BDAT
import statsmodels.api as sm
from pandas.tseries.offsets import CustomBusinessDay
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool
from tkinter import Tk, Frame, BOTH, YES
from pandastable import Table
import functools, itertools
from pandasql import PandaSQL

from .pdutils import window_mapping, year_month_from_single_date, _check_portfolio_inputs, _assert_byvars_list, \
                     _create_cutoffs_and_sort_into_ports, _split, _sort_arr_list_into_ports_and_return_series, \
                     _to_list_if_str, _expand, _to_series_if_str, _to_name_if_series, \
                     _extract_table_names_from_sql, _get_datetime_cols, \
                     _select_long_short_ports, _portfolio_difference

from .ext_time import estimate_time

def to_csv(dataframe, path, filename, output=True, action='w', index=True):
    '''
    action='w' for overwrite, 'a' for append
    set index to False to not include index in output
    '''   
    if action == 'a':
        headers = False
    else:
        headers = True
    
    if dataframe is not None: #if dataframe exists
        filepath = os.path.join(path,filename + '.csv')
        f = open(filepath, action, encoding='utf-8')
        if output is True: print("Now saving %s" % filepath)
        try: f.write(dataframe.to_csv(encoding='utf-8', index=index, header=headers)) #could use easier dataframe.to_csv(filepath) syntax, but won't overwrite
        except: f.write(dataframe.to_csv(encoding='utf-8', index=index, header=headers).replace('\ufffd',''))
        f.close()
    else:
        print("{} does not exist.".format(dataframe)) #does nothing if dataframe doesn't exist
    
def convert_sas_date_to_pandas_date(sasdates):
    epoch = datetime.datetime(1960, 1, 1)
    
    def to_pandas(date):
        return epoch + datetime.timedelta(days=date)
    
    if isinstance(sasdates, pd.Series):
        #Below code is to reduce down to unique dates and create a mapping
        
#         unique = pd.Series(sasdates.dropna().unique()).astype(int)
#         shift = unique.apply(datetime.timedelta)
#         pd_dates = epoch + shift
        
#         for_merge = pd.concat([unique, pd_dates], axis=1)
#         for_merge.columns = [sasdates.name, 0]
        
#         orig_df = pd.DataFrame(sasdates)
#         orig_df.reset_index(inplace=True)
        
#         return for_merge.merge(orig_df, how='right', on=[sasdates.name]).sort_values('index').reset_index()[0]

        return apply_func_to_unique_and_merge(sasdates, to_pandas)
    
    
#         return pd.Series([epoch + datetime.timedelta(days=int(float(date))) if not pd.isnull(date) else nan for date in sasdates])
    else:
        return epoch + datetime.timedelta(days=sasdates)
    
def year_month_from_date(df, date='Date', yearname='Year', monthname='Month'):
    '''
    Takes a dataframe with a datetime object and creates year and month variables
    '''
    df = df.copy()
#     df[yearname] =  [date.year  for date in df[date]]
#     df[monthname] = [date.month for date in df[date]]
    df[[yearname, monthname]] = apply_func_to_unique_and_merge(df[date], year_month_from_single_date)
    
    return df

def expand_time(df, intermediate_periods=False, **kwargs):
    """
    Creates new observations in the dataset advancing the time by the int or list given. Creates a new date variable.
    See _expand_time for keyword arguments.
    
    Specify intermediate_periods=True to get periods in between given time periods, e.g.
    passing time=[12,24,36] will get periods 12, 13, 14, ..., 35, 36. 
    """
    
    if intermediate_periods:
        assert 'time' in kwargs
        time = kwargs['time']
        time = [t for t in range(min(time),max(time) + 1)]
        kwargs['time'] = time
    return _expand_time(df, **kwargs)

def _expand_time(df, datevar='Date', freq='m', time=[12, 24, 36, 48, 60], newdate='Shift Date', shiftvar='Shift'):
    '''
    Creates new observations in the dataset advancing the time by the int or list given. Creates a new date variable.
    '''
    def log(message):
        if message != '\n':
            time = datetime.datetime.now().replace(microsecond=0)
            message = str(time) + ': ' + message
        sys.stdout.write(message + '\n')
        sys.stdout.flush()
    
    log('Initializing expand_time for periods {}.'.format(time))
    
    if freq == 'd':
        log('Daily frequency, getting trading day calendar.')
        td = tradedays() #gets trading day calendar
    else:
        td = None
    
    def time_shift(shift, freq=freq, td=td):
        if freq == 'm':
            return relativedelta(months=shift)
        if freq == 'd':
            return shift * td
        if freq == 'a':
            return relativedelta(years=shift)
    
    if isinstance(time, int):
        time = [time]
    else: assert isinstance(time, list)
    
    
    log('Calculating number of rows.')
    num_rows = len(df.index)
    log('Calculating number of duplicates.')
    duplicates = len(time)
    
    #Expand number of rows
    if duplicates > 1:
        log('Duplicating observations {} times.'.format(duplicates - 1))
        df = df.append([df] * (duplicates - 1)).sort_index().reset_index(drop=True)
        log('Duplicated.')
    
    log('Creating shift variable.')
    df[shiftvar] = time * num_rows #Create a variable containing amount of time to shift
    #Now create shifted date
    log('Creating shifted date.')
    df[newdate] = [date + time_shift(int(shift)) for date, shift in zip(df[datevar],df[shiftvar])]
    log('expand_time completed.')
    
    #Cleanup and exit
    return df #.drop('Shift', axis=1)

def expand_months(df, datevar='Date', newdatevar='Daily Date', trade_days=True):
    """
    Takes a monthly dataframe and returns a daily (trade day or calendar day) dataframe. 
    For each row in the input data, duplicates that row over each trading/calendar day in the month of 
    the date in that row. Creates a new date column containing the daily date.
    
    NOTE: If the input dataset has multiple observations per month, all of these will be expanded. Therefore
    you will have one row for each trade day for each original observation. 
    
    Required inputs:
    df: pandas dataframe containing a date variable
    
    Optional inputs:
    datevar: str, name of column containing dates in the input df
    newdatevar: str, name of new column to be created containing daily dates
    tradedays: bool, True to use trading days and False to use calendar days
    """
    if trade_days:
        td = tradedays()
    else:
        td = 'D'
    
    expand = functools.partial(_expand, datevar=datevar, td=td, newdatevar=newdatevar)
    
    
    expand_all = np.vectorize(expand, otypes=[np.ndarray])
        
    days =  pd.DataFrame(np.concatenate(expand_all(df[datevar].unique()), axis=0),
                         columns=[datevar, newdatevar], dtype='datetime64')

    return df.merge(days, on=datevar, how='left')

def cumulate(df, cumvars, method, periodvar='Date',  byvars=None, time=None, grossify=False,
             multiprocess=True):
    """
    Cumulates a variable over time. Typically used to get cumulative returns. 
    
    NOTE: Method zero not yet working
    
    method = 'between', 'zero', or 'first'. 
             If 'zero', will give returns since the original date. Note: for periods before the original date, 
             this will turn positive returns negative as we are going backwards in time.
             If 'between', will give returns since the prior requested time period. Note that
             the first period is period 0.
             If 'first', will give returns since the first requested time period.
             For example, if our input data was for date 1/5/2006, but we had shifted dates:
                 permno  date      RET  shift_date
                 10516   1/5/2006  110%  1/5/2006
                 10516   1/5/2006  120%  1/6/2006
                 10516   1/5/2006  105%  1/7/2006
                 10516   1/5/2006  130%  1/8/2006
             Then cumulate(df, 'RET', cumret='between', time=[1,3], get='RET', periodvar='shift_date') would return:
                 permno  date      RET  shift_date  cumret
                 10516   1/5/2006  110%  1/5/2006    110%
                 10516   1/5/2006  120%  1/6/2006    120%
                 10516   1/5/2006  105%  1/7/2006    126%
                 10516   1/5/2006  130%  1/8/2006    130%
             Then cumulate(df, 'RET', cumret='first', periodvar='shift_date') would return:
                 permno  date      RET  shift_date  cumret
                 10516   1/5/2006  110%  1/5/2006    110%
                 10516   1/5/2006  120%  1/6/2006    120%
                 10516   1/5/2006  105%  1/7/2006    126%
                 10516   1/5/2006  130%  1/8/2006    163.8%
    byvars: string or list of column names to use to seperate by groups
    time: list of ints, for use with method='between'. Defines which periods to calculate between.
    grossify: bool, set to True to add one to all variables then subtract one at the end
    multiprocess: bool or int, set to True to use all available processors, 
                  set to False to use only one, pass an int less or equal to than number of 
                  processors to use that amount of processors 
    """
    import time as time2 #accidentally used time an an input parameter and don't want to break prior code
    
    def log(message):
        if message != '\n':
            time = datetime.datetime.now().replace(microsecond=0)
            message = str(time) + ': ' + message
        sys.stdout.write(message + '\n')
        sys.stdout.flush()
    
    log('Initializing cumulate.')
    
    df = df.copy() #don't want to modify original dataframe
    
    if time:
        sort_time = sorted(time)
    else: sort_time = None
        
    if isinstance(cumvars, (str, int)):
        cumvars = [cumvars]
    assert isinstance(cumvars, list)

    assert isinstance(grossify, bool)
    
    if grossify:
        for col in cumvars:
            df[col] = df[col] + 1
    
    def unflip(df, cumvars):
        flipcols = ['cum_' + str(c) for c in cumvars] #select cumulated columns
        for col in flipcols:
            tempdf[col] = tempdf[col].shift(1) #shift all values down one row for cumvars
            tempdf[col] = -tempdf[col] + 2 #converts a positive return into a negative return
        tempdf = tempdf[1:].copy() #drop out period 0
        tempdf = tempdf.sort_values(periodvar) #resort to original order
        
    def flip(df, flip):
        flip_df = df[df['window'].isin(flip)]
        rest = df[~df['window'].isin(flip)]
        flip_df = flip_df.sort_values(byvars + [periodvar], ascending=False)
        return pd.concat([flip_df, rest], axis=0)
    
    def _cumulate(array_list, mp=multiprocess):
        if multiprocess:
            if isinstance(multiprocess, int):
                return _cumulate_mp(array_list, mp=mp) #use mp # processors
            else:
                return _cumulate_mp(array_list) #use all processors
        else:
            return _cumulate_sp(array_list)
    
    def _cumulate_sp(array_list):
        out_list = []
        for array in array_list:
            out_list.append(np.cumprod(array, axis=0))
        return np.concatenate(out_list, axis=0)
    
    def _cumulate_mp(array_list, mp=None):
        if mp:
            with Pool(mp) as pool: #use mp # processors
                return _cumulate_mp_main(array_list, pool)
        else:
            with Pool() as pool: #use all processors
                return _cumulate_mp_main(array_list, pool)
        
    def _cumulate_mp_main(array_list, pool):
        
        #For time estimation
        counter = []
        num_loops = len(array_list)
        start_time = timeit.default_timer()
        
        #Mp setup
        cum = functools.partial(np.cumprod, axis=0)
        results = [pool.apply_async(cum, (arr,), callback=counter.append) for arr in array_list]
        
        #Time estimation
        while len(counter) < num_loops:
            estimate_time(num_loops, len(counter), start_time)
            time2.sleep(0.5)
            
        #Collect and output results. A timeout of 1 should be fine because
        #it should wait until completion anyway
        return np.concatenate([r.get(timeout=1) for r in results], axis=0)
        

    def split(df, cumvars, periodvar):
        """
        Splits a dataframe into a list of arrays based on a key variable
        """
#         df = df.sort_values(['__key_var__', periodvar])
        small_df = df[['__key_var__'] + cumvars]
        arr = small_df.values
        splits = []
        for i in range(arr.shape[0]):
            if i == 0: continue
            if arr[i,0] != arr[i-1,0]: #different key
                splits.append(i)
        return np.split(arr[:,1:], splits)
    
    #####TEMPORARY CODE######
    assert method.lower() != 'zero'
    #########################
    
    if isinstance(byvars, str):
        byvars = [byvars]
    
    assert method.lower() in ('zero','between','first')
    assert not ((method.lower() == 'between') and (time == None)) #need time for between method
    if time != None and method.lower() != 'between':
        warnings.warn('Time provided but method was not between. Time will be ignored.')

    #Creates a variable containing index of window in which the observation belongs
    if method.lower() == 'between':
        df = _map_windows(df, sort_time, method=method, periodvar=periodvar, byvars=byvars)
    else:
        df['__map_window__'] = 1
        df.loc[df[periodvar] == min(df[periodvar]), '__map_window__'] = 0

        
    
    
    ####################TEMP
#     import pdb
#     pdb.set_trace()
    #######################
    
    
    if not byvars:  byvars = ['__map_window__']
    else: byvars.append('__map_window__')
    assert isinstance(byvars, list)
    
    #need to determine when to cumulate backwards
    #check if method is zero, there only negatives and zero, and there is at least one negative in each window
    if method.lower() == 'zero': 
        #flip is a list of indices of windows for which the window should be flipped
        flip = [j for j, window in enumerate(windows) \
               if all([i <= 0 for i in window]) and any([i < 0 for i in window])]
        df = flip(df, flip)
        

    log('Creating by groups.')

    #Create by groups
    df['__key_var__'] = '__key_var__' #container for key
    for col in [df[c].astype(str) for c in byvars]:
        df['__key_var__'] += col

    array_list = split(df, cumvars, periodvar)
    
#     container_array = df[cumvars].values
    full_array = _cumulate(array_list)
    
    new_cumvars = ['cum_' + str(c) for c in cumvars]

    cumdf = pd.DataFrame(full_array, columns=new_cumvars, dtype=np.float64)
    outdf = pd.concat([df.reset_index(drop=True), cumdf], axis=1)
    
    if method.lower == 'zero' and flip != []: #if we flipped some of the dataframe
        pass #TEMPORARY
    
    
    
    if grossify:
        all_cumvars = cumvars + new_cumvars
        for col in all_cumvars:
            outdf[col] = outdf[col] - 1
    
    drop_cols = [col for col in outdf.columns if col.startswith('__')]
    
    return outdf.drop(drop_cols, axis=1)

def long_to_wide(df, groupvars, values, colindex=None):
    '''
    
    groupvars = string or list of variables which signify unique observations in the output dataset
    values = string or list of variables which contain the values which need to be transposed
    colindex = variable containing extension for column name in the output dataset. If not specified, just uses the
               count of the row within the group.
    
    NOTE: Don't have any variables named key or idx
    
    For example, if we had a long dataset of returns, with returns 12, 24, 36, 48, and 60 months after the date:
            ticker    ret    months
            AA        .01    12
            AA        .15    24
            AA        .21    36
            AA       -.10    48
            AA        .22    60
    and we want to get this to one observation per ticker:
            ticker    ret12    ret24    ret36    ret48    ret60    
            AA        .01      .15      .21     -.10      .22
    We would use:
    long_to_wide(df, groupvars='ticker', values='ret', colindex='months')
    '''
    
    df = df.copy() #don't overwrite original
    
    #Check for duplicates
    if df.duplicated().any():
        df.drop_duplicates(inplace=True)
        warnings.warn('Found duplicate rows and deleted.')
    
    #Ensure type of groupvars is correct
    if isinstance(groupvars,str):
        groupvars = [groupvars]
    assert isinstance(groupvars, list)
    
    #Ensure type of values is correct
    if isinstance(values,str):
        values = [values]
    assert isinstance(values, list)
    #Use count of the row within the group for column index if not specified
    if colindex == None:
        df['__idx__'] = df.groupby(groupvars).cumcount()
        colindex = '__idx__'
    
    df['__key__'] = df[groupvars[0]].astype(str) #create key variable
    if len(groupvars) > 1: #if there are multiple groupvars, combine into one key
        for var in groupvars[1:]:
            df['__key__'] = df['__key__'] + '_' + df[var].astype(str)
    
    #Create seperate wide datasets for each value variable then merge them together
    for i, value in enumerate(values):
        if i == 0:
            combined = df.copy()
        #Create wide dataset
        raw_wide = df.pivot(index='__key__', columns=colindex, values=value)
        raw_wide.columns = [value + str(col) for col in raw_wide.columns]
        wide = raw_wide.reset_index()

        #Merge back to original dataset
        combined = combined.merge(wide, how='left', on='__key__')
    
    return combined.drop([colindex,'__key__'] + values, axis=1).drop_duplicates().reset_index(drop=True)

def load_sas(filepath, csv=True, **read_csv_kwargs):  
    sas_name = os.path.basename(filepath) #e.g. dsename.sas7bdat
    folder = os.path.dirname(filepath) #location of sas file
    filename, extension = os.path.splitext(sas_name) #returns ('dsenames','.sas7bdat')
    csv_name = filename + '.csv'
    csv_path = os.path.join(folder, csv_name)
    
    if os.path.exists(csv_path) and csv:
        if os.path.getmtime(csv_path) > os.path.getmtime(filepath): #if csv was modified more recently
            #Read from csv (don't touch sas7bdat because slower loading)
            try: return pd.read_csv(csv_path, encoding='utf-8', **read_csv_kwargs)
            except UnicodeDecodeError: return pd.read_csv(csv_path, encoding='cp1252', **read_csv_kwargs)
    
    #In the case that there is no csv already, or that the sas7bdat has been modified more recently
    #Pull from SAS file
    df = SAS7BDAT(filepath).to_data_frame()
    #Write to csv file
    if csv:
        to_csv(df, folder, filename, output=False, index=False)
    return df

def averages(df, avgvars, byvars, wtvar=None, count=False, flatten=True):
    '''
    Returns equal- and value-weighted averages of variables within groups
    
    avgvars: List of strings or string of variable names to take averages of
    byvars: List of strings or string of variable names for by groups
    wtvar: String of variable to use for calculating weights in weighted average
    count: False or string of variable name, pass variable name to get count of non-missing
           of that variable within groups.
    flatten: Boolean, False to return df with multi-level index
    '''
    #Check types
    assert isinstance(df, pd.DataFrame)
    if isinstance(avgvars, str): avgvars = [avgvars]
    else:
        assert isinstance(avgvars, list)
    assert isinstance(byvars, (str, list))
    if wtvar != None:
        assert isinstance(wtvar, str)
    
    df = df.copy()
    
    if count:
        df = groupby_merge(df, byvars, 'count', subset=count)
        avgvars += [count + '_count']
    
    g = df.groupby(byvars)
    avg_df  = g.mean()[avgvars]
    
    if wtvar == None:
        if flatten:
            return avg_df.reset_index()
        else:
            return avg_df
    
    for var in avgvars:
        colname = var + '_wavg'
        df[colname] = df[wtvar] / g[wtvar].transform('sum') * df[var]
    
    wavg_cols = [col for col in df.columns if col[-4:] == 'wavg']
    
    g = df.groupby(byvars) #recreate because we not have _wavg cols in df
    wavg_df = g.sum()[wavg_cols]
    
    outdf = pd.concat([avg_df,wavg_df], axis=1)
    
    if flatten:
        return outdf.reset_index()
    else:
        return outdf
    
def portfolio(df, groupvar, ngroups=10, byvars=None, cutdf=None, portvar='portfolio',
              multiprocess=False):
    '''
    Constructs portfolios based on percentile values of groupvar. If ngroups=10, then will form 10 portfolios,
    with portfolio 1 having the bottom 10 percentile of groupvar, and portfolio 10 having the top 10 percentile
    of groupvar.
    
    df: pandas dataframe, input data
    groupvar: string, name of variable in df to form portfolios on
    ngroups: integer, number of portfolios to form
    byvars: string, list, or None, name of variable(s) in df, finds portfolios within byvars. For example if byvars='Month',
            would take each month and form portfolios based on the percentiles of the groupvar during only that month
    cutdf: pandas dataframe or None, optionally determine percentiles using another dataset. See second note.
    portvar: string, name of portfolio variable in the output dataset
    multiprocess: bool or int, set to True to use all available processors, 
                  set to False to use only one, pass an int less or equal to than number of 
                  processors to use that amount of processors 
    
    NOTE: Resets index and drops in output data, so don't use if index is important (input data not affected)
    NOTE: If using a cutdf, MUST have the same bygroups as df. The number of observations within each bygroup
          can be different, but there MUST be a one-to-one match of bygroups, or this will NOT work correctly.
          This may require some cleaning of the cutdf first.
    NOTE: For some reason, multiprocessing seems to be slower in testing, so it is disabled by default
    '''
    #Check types
    _check_portfolio_inputs(df, groupvar, ngroups=ngroups, byvars=byvars, cutdf=cutdf, portvar=portvar)
    byvars = _assert_byvars_list(byvars)
    if cutdf != None:
        assert isinstance(cutdf, pd.DataFrame)
    else: #this is where cutdf == None, the default case
        cutdf = df
        tempcutdf = cutdf.copy()
    
    pct_per_group = 100/ngroups
    percentiles = [i*pct_per_group for i in range(ngroups)] #percentile values, e.g. 0, 10, 20, 30... 100
    percentiles += [100]
    
#     pct_per_group = int(100/ngroups)
#     percentiles = [i for i in range(0, 100 + pct_per_group, pct_per_group)] #percentile values, e.g. 0, 10, 20, 30... 100
    
    #Create new functions with common arguments added
    create_cutoffs_and_sort_into_ports = functools.partial(_create_cutoffs_and_sort_into_ports, 
                                       groupvar=groupvar, portvar=portvar, percentiles=percentiles)
    split = functools.partial(_split, keepvars=[groupvar], force_numeric=True)
    sort_arr_list_into_ports_and_return_series = functools.partial(_sort_arr_list_into_ports_and_return_series,
                                                         percentiles=percentiles,
                                                         multiprocess=multiprocess)
    
    tempdf = df.copy()
    
    #If there are no byvars, just complete portfolio sort
    if byvars == None: return create_cutoffs_and_sort_into_ports(tempdf, cutdf)
    
    #The below rename is incase there is already a variable named index in the data
    #The rename will just not do anything if there's not
    tempdf = tempdf.reset_index(drop=True).rename(
        columns={'index':'__temp_index__'}).reset_index() #get a variable 'index' containing obs count
    
    #Also replace index in byvars if there
    temp_byvars = [b if b != 'index' else '__temp_index__' for b in byvars]
    all_byvars = [temp_byvars, byvars] #list of lists
    
    #else, deal with byvars
    #First create a key variable based on all the byvars
    for i, this_df in enumerate([tempdf, tempcutdf]):
        this_df['__key_var__'] = 'key' #container for key
        for col in [this_df[c].astype(str) for c in all_byvars[i]]:
            this_df['__key_var__'] += col
        this_df.sort_values('__key_var__', inplace=True)
    
    #Now split into list of arrays and process
    array_list = split(tempdf)
    cut_array_list = split(tempcutdf)

    tempdf = tempdf.reset_index(drop=True) #need to reset index again for adding new column
    tempdf[portvar] = sort_arr_list_into_ports_and_return_series(array_list, cut_array_list)
    return tempdf.sort_values('index').drop(['__key_var__','index'], axis=1).rename(
                columns={'__temp_index__':'index'}).reset_index(drop=True)

    
def portfolio_averages(df, groupvar, avgvars, ngroups=10, byvars=None, cutdf=None, wtvar=None,
                       count=False, portvar='portfolio', avgonly=False):
    '''
    Creates portfolios and calculates equal- and value-weighted averages of variables within portfolios. If ngroups=10,
    then will form 10 portfolios, with portfolio 1 having the bottom 10 percentile of groupvar, and portfolio 10 having 
    the top 10 percentile of groupvar.
    
    df: pandas dataframe, input data
    groupvar: string, name of variable in df to form portfolios on
    avgvars: string or list, variables to be averaged
    ngroups: integer, number of portfolios to form
    byvars: string, list, or None, name of variable(s) in df, finds portfolios within byvars. For example if byvars='Month',
            would take each month and form portfolios based on the percentiles of the groupvar during only that month
    cutdf: pandas dataframe or None, optionally determine percentiles using another dataset
    wtvar: string, name of variable in df to use for weighting in weighted average
    count: False or string of variable name, pass variable name to get count of non-missing
           of that variable within groups.
    portvar: string, name of portfolio variable in the output dataset
    avgonly: boolean, True to return only averages, False to return (averages, individual observations with portfolios)
    
    NOTE: Resets index and drops in output data, so don't use if index is important (input data not affected)
    '''
    ports = portfolio(df, groupvar, ngroups=ngroups, byvars=byvars, cutdf=cutdf, portvar=portvar)
    if byvars:
        assert isinstance(byvars, (str, list))
        if isinstance(byvars, str): byvars = [byvars]
        by = [portvar] + byvars
        avgs = averages(ports, avgvars, byvars=by, wtvar=wtvar, count=count)
    else:
        avgs = averages(ports, avgvars, byvars=portvar, wtvar=wtvar, count=count)
    
    if avgonly:
        return avgs
    else:
        return avgs, ports
    
def reg_by(df, yvar, xvars, groupvar, merge=False, cons=True):
    """
    Runs a regression of df[yvar] on df[xvars] by values of groupvar. Outputs a dataframe with values of 
    groupvar and corresponding coefficients, unless merge=True, then outputs the original dataframe with the
    appropriate coefficients merged in.
    
    Required inputs:
    groupvar: str or list of strs, column names of columns identifying by groups
    
    Optional Options:
    cons: True to include a constant, False to not
    """   
    result_df = pd.DataFrame()
    
    if isinstance(xvars, str):
        xvars = [xvars]
    assert isinstance(xvars, list)
    
    drop_group = False
    if isinstance(groupvar, list):
        df['__key_regby__'] = ''
        for var in groupvar:
            df['__key_regby__'] = df['__key_regby__'] + df[var].astype(str)
        groupvar = '__key_regby__'
        drop_group = True
    
    for group in df[groupvar].unique():
        tempdf = df[df[groupvar] == group][xvars + [yvar]].dropna() #will fail with nans
        X = tempdf[xvars]
        
        if cons:
            X = sm.add_constant(X)
        
        y = tempdf[yvar]

        if len(tempdf.index) > len(xvars) + 1: #if enough observations, run regression
            model = sm.OLS(y, X)
            result = model.fit()
            this_result = pd.DataFrame(result.params).T
        else: #not enough obs, return nans 
            if cons:
                rhs = ['const'] + xvars 
            else:
                rhs = xvars
            this_result = pd.DataFrame([tuple([nan for col in rhs])], columns=rhs)
        
        this_result[groupvar] = group
        result_df = result_df.append(this_result) #  Or whatever summary info you want
    
    result_df.columns = ['coef_' + col if col not in (groupvar, 'const') else col for col in result_df.columns]
    
    if merge:
        out = df.merge(result_df, how='left', on=groupvar)
        if drop_group:
            out.drop(groupvar, axis=1, inplace=True)
        return out
    
    return result_df.reset_index(drop=True)

def factor_reg_by(df, groupvar, fac=4, retvar='RET'):
    """
    Takes a dataframe with RET, mktrf, smb, hml, and umd, and produces abnormal returns by groups.
    
    Required inputs:
    df: pandas datafram containing mktrf, smb, hml, umd, (or what's required for chosen model)
        and a return variable
    groupvar: str or list of strs, column names of columns on which to form by groups
    fac: int (1, 3, 4), factor model to run
    retvar: str, name of column containing returns
    """
    assert fac in (1, 3, 4)
    factors = ['mktrf']
    if fac >= 3:
        factors += ['smb','hml']
    if fac == 4:
        factors += ['umd']
        
#     factor_loadings = reg_by(df, 'RET', factors, groupvar)
#     outdf = df.merge(factor_loadings, how='left', on=groupvar) #merge back to sample
    outdf = reg_by(df, retvar, factors, groupvar, merge=True)
    outdf['AB' + retvar] = outdf[retvar] - sum([outdf[fac] * outdf['coef_' + fac] for fac in factors]) #create abnormal returns
    return outdf

def state_abbrev(df, col, toabbrev=False):
    df = df.copy()
    states_to_abbrev = {
    'Alabama': 'AL', 
    'Montana': 'MT',
    'Alaska': 'AK', 
    'Nebraska': 'NE',
    'Arizona': 'AZ', 
    'Nevada': 'NV',
    'Arkansas': 'AR', 
    'New Hampshire': 'NH',
    'California': 'CA', 
    'New Jersey': 'NJ',
    'Colorado': 'CO', 
    'New Mexico': 'NM',
    'Connecticut': 'CT', 
    'New York': 'NY',
    'Delaware': 'DE', 
    'North Carolina': 'NC',
    'Florida': 'FL', 
    'North Dakota': 'ND',
    'Georgia': 'GA', 
    'Ohio': 'OH',
    'Hawaii': 'HI', 
    'Oklahoma': 'OK',
    'Idaho': 'ID', 
    'Oregon': 'OR',
    'Illinois': 'IL', 
    'Pennsylvania': 'PA',
    'Indiana': 'IN', 
    'Rhode Island': 'RI',
    'Iowa': 'IA', 
    'South Carolina': 'SC',
    'Kansas': 'KS', 
    'South Dakota': 'SD',
    'Kentucky': 'KY', 
    'Tennessee': 'TN',
    'Louisiana': 'LA', 
    'Texas': 'TX',
    'Maine': 'ME', 
    'Utah': 'UT',
    'Maryland': 'MD', 
    'Vermont': 'VT',
    'Massachusetts': 'MA', 
    'Virginia': 'VA',
    'Michigan': 'MI', 
    'Washington': 'WA',
    'Minnesota': 'MN', 
    'West Virginia': 'WV',
    'Mississippi': 'MS', 
    'Wisconsin': 'WI',
    'Missouri': 'MO', 
    'Wyoming': 'WY', }
    if toabbrev:
        df[col] = df[col].replace(states_to_abbrev)
    else:
        abbrev_to_states = dict ( (v,k) for k, v in states_to_abbrev.items() )
        df[col] = df[col].replace(abbrev_to_states)
    
    return df

def create_not_trade_days(tradedays_path= r'C:\Users\derobertisna.UFAD\Desktop\Data\Other SAS\tradedays.sas7bdat'):
    df = dero.load_sas(tradedays_path)
    trading_days = pd.to_datetime(df['date']).tolist()
    all_days = pd.date_range(start=trading_days[0],end=trading_days[-1]).tolist()
    notrade_days = [day for day in all_days if day not in trading_days]
    
    outdir = os.path.dirname(tradedays_path)
    outpath = os.path.join(outdir, 'not tradedays.csv')
    
    with open(outpath, 'w') as f:
        f.write('date\n')
        f.write('\n'.join([day.date().isoformat() for day in notrade_days]))
        
def tradedays(notradedays_path=r'C:\Users\derobertisna.UFAD\Desktop\Data\Other SAS\not tradedays.csv'):
    notrade_days = pd.read_csv(notradedays_path)['date'].tolist()
    return CustomBusinessDay(holidays=notrade_days)

def select_rows_by_condition_on_columns(df, cols, condition='== 1', logic='or'):
    """
    Selects rows of a pandas dataframe by evaluating a condition on a subset of the dataframe's columns.
    
    df: pandas dataframe
    cols: list of column names, the subset of columns on which to evaluate conditions
    condition: string, needs to contain comparison operator and right hand side of comparison. For example,
               '== 1' checks for each row that the value of each column is equal to one.
    logic: 'or' or 'and'. With 'or', only one of the columns in cols need to match the condition for the row to be kept.
            With 'and', all of the columns in cols need to match the condition.
    """
    #First eliminate spaces in columns, this method will not work with spaces
    new_cols = [col.replace(' ','_').replace('.','_') for col in cols]
    df.rename(columns={col:new_col for col, new_col in zip(cols, new_cols)}, inplace=True)
    
    #Now create a string to query the dataframe with
    logic_spaces = ' ' + logic + ' '
    query_str = logic_spaces.join([str(col) + condition for col in new_cols]) #'col1 == 1, col2 == 1', etc.
    
    #Query dataframe
    outdf = df.query(query_str).copy()
    
    #Rename columns back to original
    outdf.rename(columns={new_col:col for col, new_col in zip(cols, new_cols)}, inplace=True)
    
    return outdf

def show_df(df):
    pool = ThreadPool(1)
    pool.apply_async(_show_df, args=[df])
    
def _show_df(df):
    root = Tk()
    frame = Frame(root)
    frame.pack(fill=BOTH, expand=YES)
    pt = Table(parent=frame, dataframe=df)
    pt.show()
    pt.queryBar()
    root.mainloop()
    
def groupby_merge(df, byvars, func_str, *func_args, subset='all', replace=False):
    """
    Creates a pandas groupby object, applies the aggregation function in func_str, and merges back the 
    aggregated data to the original dataframe.
    
    Required Inputs:
    df: Pandas DataFrame
    byvars: str or list, column names which uniquely identify groups
    func_str: str, name of groupby aggregation function such as 'min', 'max', 'sum', 'count', etc.
    
    Optional Input:
    subset: str or list, column names for which to apply aggregation functions
    func_args: tuple, arguments to pass to func
    replace: bool, True to replace original columns in the data with aggregated/transformed columns
    
    Usage:
    df = groupby_merge(df, ['PERMNO','byvar'], 'max', subset='RET')
    """
    
    #Convert byvars to list if neceessary
    if isinstance(byvars, str):
        byvars = [byvars]
    
    #Store all variables except byvar in subset if subset is 'all'
    if subset == 'all':
        subset = [col for col in df.columns if col not in byvars]
        
    #Convert subset to list if necessary
    if isinstance(subset, str):
        subset = [subset]
    
    if func_str == 'transform':
        #transform works very differently from other aggregation functions
        
        #First we need to deal with nans in the by variables. If there are any nans, transform will error out
        #Therefore we must fill the nans in the by variables beforehand and replace afterwards
        df[byvars] = df[byvars].fillna(value='__tempnan__')
        
        #Now we must deal with nans in the subset variables. If there are any nans, tranform will error out
        #because it tries to ignore the nan. Therefore we must remove these rows from the dataframe,
        #transform, then add those rows back.
        any_nan_subset_mask = pd.Series([all(i) for i in \
                                        (zip(*[~pd.isnull(df[col]) for col in subset]))],
                                        index=df.index)
        no_nans = df[any_nan_subset_mask]
        
        grouped = no_nans.groupby(byvars)
        func = getattr(grouped, func_str) #pull method of groupby class with same name as func_str
        grouped = func(*func_args)[subset] #apply the class method and select subset columns
        grouped.columns = [col + '_' + func_str for col in grouped.columns] #rename transformed columns
        
        df.replace('__tempnan__', nan, inplace=True) #fill nan back into dataframe
        
        #Put nan rows back
        grouped = grouped.reindex(df.index)
        
        full = pd.concat([df, grouped], axis=1)
        
    else: #.min(), .max(), etc.
        
        
        
#         grouped = df.groupby(byvars, as_index=False)[byvars + subset]
#         func = getattr(grouped, func_str) #pull method of groupby class with same name as func_str
#         grouped = func(*func_args) #apply the class method


        grouped = df.groupby(byvars)[subset]
        func = getattr(grouped, func_str) #pull method of groupby class with same name as func_str
        grouped = func(*func_args) #apply the class method
        grouped = grouped.reset_index()
        
        
        #Merge and output
        full = df.merge(grouped, how='left', on=byvars, suffixes=['','_' + func_str])
    
    if replace:
        _replace_with_transformed(full, func_str)
    
    return full
    
def _replace_with_transformed(df, func_str='transform'):
    transform_cols = [col for col in df.columns if col.endswith('_' + func_str)]
    orig_names = [col[:col.find('_' + func_str)] for col in transform_cols]
    df.drop(orig_names, axis=1, inplace=True)
    df.rename(columns={old: new for old, new in zip(transform_cols, orig_names)}, inplace=True)
    
def groupby_index(df, byvars, sortvars=None, ascending=True):
    """
    Returns a dataframe which is a copy of the old one with an additional column containing an index
    by groups. Each time the bygroup changes, the index restarts at 0.
    
    Required inputs:
    df: pandas DataFrame
    byvars: str or list of column names containing group identifiers
    
    Optional inputs:
    sortvars: str or list of column names to sort by within by groups
    ascending: bool, direction of sort
    """
    
    #Convert sortvars to list if necessary
    if isinstance(sortvars, str):
        sortvars = [sortvars]
    if sortvars == None: sortvars = []
    
    df = df.copy() #don't modify the original dataframe
    df.sort_values(byvars + sortvars, inplace=True, ascending=ascending)
    df['__temp_cons__'] = 1
    df = groupby_merge(df, byvars, 'transform', (lambda x: [i for i in range(len(x))]), subset=['__temp_cons__'])
    df.drop('__temp_cons__', axis=1, inplace=True)
    return df.rename(columns={'__temp_cons___transform': 'group_index'})

def to_copy_paste(df, index=False, column_names=True):
    """
    Takes a dataframe and prints all of its data in such a format that it can be copy-pasted to create
    a new dataframe from the pandas.DataFrame() constructor.
    
    Required inputs:
    df: pandas dataframe
    
    Optional inputs:
    index: bool, True to include index
    column_names: bool, False to exclude column names
    """
    print('pd.DataFrame(data = [')
    for tup in df.iterrows():        
        data = tup[1].values
        print(str(tuple(data)) + ',')
    last_line = ']'
    if column_names:
        last_line += ', columns = {}'.format([i for i in df.columns]) #list comp to remove Index() around cols
    if index:
        last_line += ',\n index = {}'.format([i for i in df.index]) #list comp to remove Index() around index
    last_line += ')' #end command
    print(last_line)
    
def _join_col_strings(*args):
    strs = [str(arg) for arg in args]
    return '_'.join(strs)

def join_col_strings(df, cols):
    """
    Takes a dataframe and column name(s) and concatenates string versions of the columns with those names.
    Useful for when a group is identified by several variables and we need one key variable to describe a group.
    Returns a pandas Series.
    
    Required inputs:
    df: pandas dataframe
    cols: str or list, names of columns in df to be concatenated
    """
    
    if isinstance(cols, str):
        cols = [cols]
    assert isinstance(cols, list)
    
    jc = np.vectorize(_join_col_strings)
    
    return pd.Series(jc(*[df[col] for col in cols]))

def winsorize(df, pct, subset=None, byvars=None, bot=True, top=True):
    """
    Finds observations above the pct percentile and replaces the with the pct percentile value.
    Does this for all columns, or the subset given by subset
    
    Required inputs:
    df: Pandas dataframe
    pct: 0 < float < 1 or list of two values 0 < float < 1. If two values are given, the first
         will be used for the bottom percentile and the second will be used for the top. If one value
         is given and both bot and top are True, will use the same value for both.
    
    Optional inputs:
    subset: List of strings or string of column name(s) to winsorize
    byvars: str, list of strs, or None. Column names of columns identifying groups in the data.
            Winsorizing will be done within those groups.
    bot: bool, True to winsorize bottom observations
    top: bool, True to winsorize top observations
    
    Example usage:
    winsorize(df, .05, subset='RET') #replaces observations of RET below the 5% and above the 95% values
    winsorize(df, [.05, .1], subset='RET') #replaces observations of RET below the 5% and above the 90% values

    """
    
    #Check inputs
    assert any([bot, top]) #must winsorize something
    if isinstance(pct, float):
        bot_pct = pct
        top_pct = 1 - pct
    elif isinstance(pct, list):
        bot_pct = pct[0]
        top_pct = 1 - pct[1]
    else:
        raise ValueError('pct must be float or a list of two floats')
        
    def temp_winsor(col):
        return _winsorize(col, top_pct, bot_pct, top=top, bot=bot)

    #Save column order
    cols = df.columns
    
    #Get a dataframe of data to be winsorized, and a dataframe of the other columns
    to_winsor, rest = _select_numeric_or_subset(df, subset, extra_include=byvars)

    #Now winsorize
    if byvars: #use groupby to process groups individually
        to_winsor = groupby_merge(to_winsor, byvars, 'transform', (temp_winsor), replace=True)
    else: #do entire df, one column at a time
        to_winsor.apply(temp_winsor, axis=0)
    
    return pd.concat([to_winsor,rest], axis=1)[cols]


def _winsorize(col, top_pct, bot_pct, top=True, bot=True):
    """
    Winsorizes a pandas Series
    """
    if top:
        top_val = col.quantile(top_pct)
        col.loc[col > top_val] = top_val
    if bot:
        bot_val = col.quantile(bot_pct)
        col.loc[col < bot_val] = bot_val
    return col
            
def _select_numeric_or_subset(df, subset, extra_include=None):
    """
    If subset is not None, selects all numeric columns. Else selects subset. 
    If extra_include is not None and subset is None, will select all numeric columns plus 
    those in extra_include.
    Returns a tuple of (dataframe containing subset columns, dataframe of other columns)
    """
    if subset == None:
        to_winsor = df.select_dtypes(include=[np.number, np.int64]).copy()
        subset    = to_winsor.columns
        rest      = df.select_dtypes(exclude=[np.number, np.int64]).copy()
    else:
        if isinstance(subset, str):
            subset = [subset]
        assert isinstance(subset, list)
        to_winsor = df[subset].copy()
        other_cols = [col for col in df.columns if col not in subset]
        rest = df[other_cols].copy()
    if extra_include:
        to_winsor = pd.concat([to_winsor, df[extra_include]], axis=1)
        rest.drop(extra_include, axis=1, inplace=True)
        
    return (to_winsor, rest)

def apply_func_to_unique_and_merge(series, func):
    """
    Many Pandas functions can be slow because they're doing repeated work. This function reduces
    the given series down to unique values, applies the function, then expands back up to the
    original shape of the data. Returns a series.
    
    Required inputs:
    seres: pd.Series
    func: function to be applied to the series.
    
    Usage:
    import functools
    to_datetime = functools.partial(pd.to_datetime, format='%Y%m')
    apply_func_to_unique_and_merge(df['MONTH'], to_datetime)
    """

    unique = pd.Series(series.dropna().unique())
    new = unique.apply(func)

    for_merge = pd.concat([unique, new], axis=1)
    num_cols = [i for i in range(len(for_merge.columns) - 1)] #names of new columns
    for_merge.columns = [series.name] + num_cols

    orig_df = pd.DataFrame(series)
    orig_df.reset_index(inplace=True)

    return for_merge.merge(orig_df, how='right', on=[series.name]).sort_values('index').reset_index().loc[:,num_cols]


def _map_windows(df, time, method='between', periodvar='Shift Date', byvars=['PERMNO','Date']):
    """
    Returns the dataframe with an additional column __map_window__ containing the index of the window 
    in which the observation resides. For example, if the windows are
    [[1],[2,3]], and the periods are 1/1/2000, 1/2/2000, 1/3/2000 for PERMNO 10516 with byvar
    'a', the df rows would be as follows:
         (10516, 'a', '1/1/2000', 0),
         (10516, 'a', '1/2/2000', 1),
         (10516, 'a', '1/3/2000', 1),
    """

    df = df.copy() #don't overwrite original dataframe
    
    wm = functools.partial(window_mapping, time, method=method)

    df = groupby_merge(df, byvars, 'transform', (wm), subset=periodvar)

    return df.rename(columns={periodvar + '_transform': '__map_window__'})

def left_merge_latest(df, df2, on, left_datevar='Date', right_datevar='Date',
                      limit_years=False, backend='pandas'):
    """
    Left merges df2 to df using on, but grabbing the most recent observation (right_datevar will be
    the soonest earlier than left_datevar). Useful for situations where data needs to be merged with
    mismatched dates, and just the most recent data available is needed. 
    
    Required inputs:
    df: Pandas dataframe containing source data (all rows will be kept), must have on variables
        and left_datevar
    df2: Pandas dataframe containing data to be merged (only the most recent rows before source
        data will be kept)
    on: str or list of strs, names of columns on which to match, excluding date
    
    Optional inputs:
    left_datevar: str, name of date variable on which to merge in df
    right_datevar: str, name of date variable on which to merge in df2
    limit_years: False or int, only applicable for backend='sql'. 
    backend: str, 'pandas' or 'sql'. Specify the underlying machinery used to perform the merge.
             'pandas' means native pandas, while 'sql' uses pandasql. Try 'sql' if you run
             out of memory.
    
    """
    if isinstance(on, str):
        on = [on]
        
    if backend.lower() in ('pandas','pd'):
        return _left_merge_latest_pandas(df, df2, on, left_datevar=left_datevar, right_datevar=right_datevar)
    elif backend.lower() in ('sql','pandasql'):
        return _left_merge_latest_sql(df, df2, on, left_datevar=left_datevar, right_datevar=right_datevar)
    else:
        raise ValueError("select backend='pandas' or backend='sql'.")
        
    
def _left_merge_latest_pandas(df, df2, on, left_datevar='Date', right_datevar='Date'):
    many = df.loc[:,on + [left_datevar]].merge(df2, on=on, how='left')
    
    rename = False
    #if they are named the same, pandas will automatically add _x and _y to names
    if left_datevar == right_datevar: 
        rename = True #will need to rename the _x datevar for the last step
        orig_left_datevar = left_datevar
        left_datevar += '_x'
        right_datevar += '_y'
    
    lt = many.loc[many[left_datevar] >= many[right_datevar]] #left with datadates less than date

    #find rows within groups which have the maximum right_datevar (soonest before left_datevar)
    data_rows = lt.groupby(on + [left_datevar], as_index=False)[right_datevar].max() \
        .merge(lt, on=on + [left_datevar, right_datevar], how='left')
        
    if rename: #remove the _x for final merge
        data_rows.rename(columns={left_datevar: orig_left_datevar}, inplace=True)
        return df.merge(data_rows, on=on + [orig_left_datevar], how='left')
    
    #if no renaming is required, just merge and exit
    return df.merge(data_rows, on=on + [left_datevar], how='left')

def _left_merge_latest_sql(df, df2, on, left_datevar='Date', right_datevar='Date'):
    
    if left_datevar == right_datevar:
        df2 = df2.copy()
        df2.rename(columns={right_datevar: right_datevar + '_y'}, inplace=True)
        right_datevar += '_y'
    
    on_str = ' and \n    '.join(['a.{0} = b.{0}'.format(i) for i in on])
    groupby_str = ', '.join(on)
    a_cols = ', '.join(['a.' + col for col in on + [left_datevar]])
    b_cols = ', '.join(['b.' + col for col in df2.columns if col not in on])
    query = """
    select {5}, {4}
    from
        df a
    left join
        df2 b
    on
        {0} and
        a.{1} >= b.{2}
    group by a.{3}, a.{1}
    having
        b.{2} = max(b.{2})
    """.format(on_str, left_datevar, right_datevar, groupby_str, b_cols, a_cols)
    
    return df.merge(sql([df, df2], query), on=on + [left_datevar], how='left')
    

def var_change_by_groups(df, var, byvars, datevar='Date', numlags=1):
    """
    Used for getting variable changes over time within bygroups. 
    
    NOTE: Dataset is not sorted in this process. Sort the data in the order in which you wish
          lags to be created before running this command.
    
    Required inputs:
    df: pandas dataframe containing bygroups, a date variable, and variables of interest
    var: str or list of strs, column names of variables to get changes
    byvars: str or list of strs, column names of variables identifying by groups
    
    Optional inputs:
    datevar: str ot list of strs, column names of variables identifying periods
    numlags: int, number of periods to go back to get change
    """
    var, byvars, datevar = [_to_list_if_str(v) for v in [var, byvars, datevar]] #convert to lists
    short_df = df.loc[~pd.isnull(df[byvars]).any(axis=1), var + byvars + datevar].drop_duplicates()
    for v in var:
        short_df[v + '_lag'] = short_df.groupby(byvars)[v].shift(numlags)
        short_df[v + '_change'] = short_df[v] - short_df[v + '_lag']
    dropvars = [v for v in var] + [v  + '_lag' for v in var]
    short_df = short_df.drop(dropvars, axis=1)
    return df.merge(short_df, on=datevar + byvars, how='left')

def fill_excluded_rows(df, byvars, fillvars=None, **fillna_kwargs):
    """
    Takes a dataframe which does not contain all possible combinations of byvars as rows. Creates
    those rows if fillna_kwargs are passed, calls fillna using fillna_kwargs for fillvars.
    
    For example, df:
                 date     id  var
        0  2003-06-09 42223C    1
        1  2003-06-10 09255G    2
    with fillna_for_excluded_rows(df, byvars=['date','id'], fillvars='var', value=0) becomes:
                  date     id  var
        0  2003-06-09 42223C    1
        1  2003-06-10 42223C    0
        2  2003-06-09 09255G    0
        3  2003-06-10 09255G    2
        
    Required options:
    df: pandas dataframe
    byvars: variables on which dataset should be expanded to product. Can pass a str, list of 
            strs, or a list of pd.Series.
    
    Optional options:
    fillvars: variables to apply fillna to
    fillna_kwargs: See pandas.DataFrame.fillna for kwargs, value=0 is common
    
    
    """
    byvars, fillvars = [_to_list_if_str(v) for v in [byvars, fillvars]] #convert to lists
    
    
#     multiindex = [df[i].dropna().unique() for i in byvars]
    multiindex = [_to_series_if_str(df, i).dropna().unique() for i in byvars]
    byvars = [_to_name_if_series(i) for i in byvars] #get name of any series


    all_df = pd.DataFrame(index=pd.MultiIndex.from_product(multiindex)).reset_index()
    all_df.columns = byvars
    merged = all_df.merge(df, how='left', on=byvars)
    
    if fillna_kwargs:
        fillna_kwargs.update({'inplace':False})
        merged[fillvars] = merged[fillvars].fillna(**fillna_kwargs)
    return merged

def sql(df_list, query):
    """
    Convenience function for running a pandasql query. Keeps track of which variables are of
    datetime type, and converts them back after running the sql query.
    
    NOTE: Ensure that dfs are passed in the order that they are used in the query.
    """
    #Pandasql looks up tables by names given in query. Here we are passed a list of dfs without names.
    #Therefore we need to extract the names of the tables from the query, then assign 
    #those names to the dfs in df_list in the locals dictionary.
    table_names = _extract_table_names_from_sql(query)
    for i, name in enumerate(table_names):
        locals().update({name: df_list[i]})
    
    #Get date variable column names
    datevars = []
#     othervars = []
    for d in df_list:
        datevars += _get_datetime_cols(d)
#         othervars += [col for col in d.columns if col not in datevars]
    datevars = list(set(datevars)) #remove duplicates
#     othervars = list(set(othervars))
    
    merged = PandaSQL()(query)
    
    #Convert back to datetime
    for date in [d for d in datevars if d in merged.columns]:
        merged[date] = pd.to_datetime(merged[date])
    return merged

def long_short_portfolio(df, portvar, byvars=None, retvars=None, top_minus_bot=True):
    """
    Takes a df with a column of numbered portfolios and creates a new
    portfolio which is long the top portfolio and short the bottom portfolio. 
    Returns a df of long-short portfolio
    
    Required inputs:
    df: pandas dataframe containing a column with portfolio numbers
    portvar: str, name of column containing portfolios
    
    Optional inputs:
    byvars: str or list of strs of column names containing groups for portfolios.
            Calculates long-short within these groups. These should be the same groups
            in which portfolios were formed.
    retvars: str or list of strs of variables to return in the long-short dataset. 
            By default, will use all numeric variables in the df.
    top_minus_bot: boolean, True to be long the top portfolio, short the bottom portfolio.
                   False to be long the bottom portfolio, short the top portfolio.
    """
    long, short = _select_long_short_ports(df, portvar, top_minus_bot=top_minus_bot)
    return _portfolio_difference(df, portvar, long, short, byvars=byvars, retvars=retvars)

Overwriting ext_pandas.py


# Pandas Utilities

These are functions which support other functions but should not be available to users

In [9]:
%%writefile pdutils.py

import pandas as pd
import numpy as np
from itertools import chain
from collections import OrderedDict
from functools import partial
from multiprocessing import Pool
from dateutil.relativedelta import relativedelta
import timeit
import time, datetime
import re

from .ext_time import estimate_time
from .core import OrderedSet

def _to_list_if_str(var):
    if isinstance(var, str):
        return [var]
    else:
        return var

def _to_series_if_str(df, i):
    if isinstance(i, pd.Series):
        s = i
    elif isinstance(i, str):
        s = df[i]
    else:
        raise ValueError('Please provide a str, list of strs, or a list of pd.Series for byvars')
    return s

def _to_name_if_series(i):
    if isinstance(i, pd.Series):
        return i.name
    else:
        return i


def create_windows(periods, time, method='between'):

    if method.lower() == 'first':
        windows = [[0]]
        windows += [[i for i in range(1, len(periods))]]
        return windows
    elif method.lower() == 'between':
        time = [t - time[0] for t in time] #shifts time so that first period is period 0
        windows = [[0]]
        t_bot = 0
        for i, t in enumerate(time): #pick each element of time
            if t == 0: continue #already added zero
            windows.append([i for i in range(t_bot + 1, t + 1)])
            t_bot = t
        #The last window is all the leftover periods after finishing time
        extra_windows = [[i for i, per in enumerate(periods) if i not in chain.from_iterable(windows)]]
        if extra_windows != [[]]: #don't want to add empty window
            windows += extra_windows
        return windows
    
def window_mapping(time, col, method='between'):
    """
    Takes a pandas series of dates as inputs, calculates windows, and returns a series of which
    windows each observation are in. To be used with groupby.transform()
    """
    windows = create_windows(col, time, method=method)
    return [n for i in range(len(col.index)) for n, window in enumerate(windows) if i in window]


def year_month_from_single_date(date):
    d = OrderedDict()
    d.update({'Year': date.year})
    d.update({'Month': date.month})
    return pd.Series(d)


############Portfolio Utilities###############
def _check_portfolio_inputs(*args, **kwargs):
    assert isinstance(args[0], pd.DataFrame)
    assert isinstance(args[1], str)
    assert isinstance(kwargs['ngroups'], int)
    
def _assert_byvars_list(byvars):
    if byvars != None:
        if isinstance(byvars, str): byvars = [byvars]
        else:
            assert isinstance(byvars, list)
    return byvars

def _sort_into_ports(df, cutoffs, portvar, groupvar):
        df[portvar] = 0
        for i, (low_cut, high_cut) in enumerate(zip(cutoffs[:-1],cutoffs[1:])):
                rows = df[(df[groupvar] >= low_cut) & (df[groupvar] <= high_cut)].index
                df.loc[rows, portvar] = i + 1
        return df
    
def _create_cutoffs(cutdf, groupvar, percentiles):
    return [np.nanpercentile(cutdf[groupvar], i) for i in percentiles]

def _create_cutoffs_and_sort_into_ports(df, cutdf, groupvar, portvar, percentiles):
    cutoffs = _create_cutoffs(cutdf, groupvar, percentiles)
    return _sort_into_ports(df, cutoffs, portvar, groupvar)

def _split(df, keepvars, force_numeric=False):
    """
    Splits a dataframe into a list of arrays based on a key variable. Pass keepvars
    to keep variables other than the key variable
    """
#     df = df.sort_values('__key_var__') #now done outside of function
    small_df = df[['__key_var__'] + keepvars]
    arr = small_df.values
    splits = []
    for i in range(arr.shape[0]):
        if i == 0: continue
        if arr[i,0] != arr[i-1,0]: #different key
            splits.append(i)
    outarr = arr[:,1:]
    if force_numeric:
        outarr = outarr.astype('float64')
    return np.split(outarr, splits)


def _create_cutoffs_arr(arr, percentiles):
    arr = arr[~np.isnan(arr)]
    if arr.size == 0:
        return False
    return [np.percentile(arr, i) for i in percentiles]


def _sort_arr_into_ports(arr, cutoffs):
    port_cutoffs = _gen_port_cutoffs(cutoffs)
    portfolio_match = partial(_portfolio_match, port_cutoffs=port_cutoffs)
    return [portfolio_match(elem) for elem in arr]


def _portfolio_match(elem, port_cutoffs):
    if np.isnan(elem): return 0
    return [index + 1 for index, bot, top in port_cutoffs \
            if elem >= bot and elem <= top][0]
    
def _gen_port_cutoffs(cutoffs):
    return [(i, low_cut, high_cut) \
                    for i, (low_cut, high_cut) \
                    in enumerate(zip(cutoffs[:-1],cutoffs[1:]))]


def _sort_arr_list_into_ports(array_list, cut_array_list, percentiles, multiprocess):
    if multiprocess:
        if isinstance(multiprocess, int):
            return _sort_arr_list_into_ports_mp(array_list, cut_array_list, percentiles, mp=multiprocess)
        else:
            return _sort_arr_list_into_ports_mp(array_list, cut_array_list, percentiles)
    else:
        return _sort_arr_list_into_ports_sp(array_list, cut_array_list, percentiles)


def _create_cutoffs_arr_and_sort_into_ports(data_tup, percentiles):
    arr, cutarr = data_tup
    cutoffs = _create_cutoffs_arr(cutarr, percentiles)
    if cutoffs:
        return _sort_arr_into_ports(arr, cutoffs)
    else:
        return [0 for elem in arr]
    
def _sort_arr_list_into_ports_sp(array_list, cut_array_list, percentiles):
    outlist = []
    for i, arr in enumerate(array_list):
        result = _create_cutoffs_arr_and_sort_into_ports((arr, cut_array_list[i]),
                                                        percentiles)
        outlist.append(result)
    return outlist

def _sort_arr_list_into_ports_mp(array_list, cut_array_list, percentiles, mp=None):
    if mp:
        with Pool(mp) as pool: #use mp # processors
            return _sort_arr_list_into_ports_mp_main(array_list, cut_array_list, percentiles, pool)
    else:
        with Pool() as pool: #use all processors
            return _sort_arr_list_into_ports_mp_main(array_list, cut_array_list, percentiles, pool)

    
def _sort_arr_list_into_ports_mp_main(array_list, cut_array_list, percentiles, pool):
        #For time estimation
        counter = []
        num_loops = len(array_list)
        start_time = timeit.default_timer()
        
        #Mp setup
        port = partial(_create_cutoffs_arr_and_sort_into_ports,
                                 percentiles=percentiles)
        
        data_tups = [(arr, cut_array_list[i]) for i, arr in enumerate(array_list)]

        results = [pool.apply_async(port, ((arr, cut_array_list[i]),), callback=counter.append) \
                        for i, arr in enumerate(array_list)]
        
        #Time estimation
        while len(counter) < num_loops:
            estimate_time(num_loops, len(counter), start_time)
            time.sleep(0.5)
            
        #Collect and output results. A timeout of 1 should be fine because
        #it should wait until completion anyway
        return [r.get(timeout=1) for r in results]


def _arr_list_to_series(array_list):
    return pd.Series(np.concatenate(array_list, axis=0))

def _sort_arr_list_into_ports_and_return_series(array_list, cut_array_list, percentiles, multiprocess):
    al = _sort_arr_list_into_ports(array_list, cut_array_list, percentiles, multiprocess)
    return _arr_list_to_series(al)

#############End portfolio utilities###########################################################################

def _expand(monthly_date, datevar, td, newdatevar):
   
    t = time.gmtime(monthly_date/1000000000) #date coming in as integer, need to parse
    t = datetime.date(t.tm_year, t.tm_mon, t.tm_mday) #better output than gmtime
    
    beginning = datetime.date(t.year, t.month, 1) #beginning of month of date
    end = beginning + relativedelta(months=1, days=-1) #last day of month
    days = pd.date_range(start=beginning, end=end, freq=td) #trade days within month
    days.name = newdatevar
    result =  np.array([(t, i) for i in days])
    return result


def _extract_table_names_from_sql(query):
    """ Extract table names from an SQL query. """
    # a good old fashioned regex. turns out this worked better than actually parsing the code
    tables_blocks = re.findall(r'(?:FROM|JOIN)\s+(\w+(?:\s*,\s*\w+)*)', query, re.IGNORECASE)
    tables = [tbl
              for block in tables_blocks
              for tbl in re.findall(r'\w+', block)]
    return OrderedSet(tables)

def _get_datetime_cols(df):
    """
    Returns a list of column names of df for which the dtype starts with datetime
    """
    dtypes = df.dtypes
    return dtypes.loc[dtypes.apply(lambda x: str(x).startswith('datetime'))].index.tolist()

def _select_long_short_ports(df, portvar, top_minus_bot=True):
    """
    Finds the appropriate portfolio number and returns (long number, short number)
    """
    #Get numbered value of highest and lowest portfolio
    top = max(df[portvar])
    bot = min(df[portvar])
    
    if top_minus_bot:
        return top, bot
    else:
        return bot, top
    
def _portfolio_difference(df, portvar, long, short, byvars=None, retvars=None):
    """
    Calculates long portfolio minus short portfolio
    """
    if byvars:
        out = df[df[portvar] == long].set_index(byvars) - df[df[portvar] == short].set_index(byvars)
    else:
        out = df[df[portvar] == long] - df[df[portvar] == short]
        
    if retvars:
        return out[retvars]
    else:
        return out


Overwriting pdutils.py


# Regressions

In [10]:
%%writefile reg.py

import itertools
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

def reg(df, yvar, xvars, robust=True, cluster=False):
    """
    Returns a fitted regression. Takes df, produces a regression df with no missing among needed
    variables, and fits a regression model. If robust is specified, uses heteroskedasticity-
    robust standard errors. If cluster is specified, calculated clustered standard errors
    by the given variable. 
    
    Note: only specify at most one of robust and cluster.
    
    Required inputs:
    df: pandas dataframe containing regression data
    yvar: str, column name of outcome y variable
    xvars: list of strs, column names of x variables for regression
    
    Optional inputs:
    robust: bool, set to True to use heterskedasticity-robust standard errors
    cluster: False or str, set to a column name to calculate standard errors within clusters
             given by unique values of given column name
    """
    drop_set = [yvar] + xvars
    if cluster:
        drop_set += [cluster]
    
    regdf = df.dropna(subset=drop_set)
    y = regdf[yvar]
    X = regdf.loc[:, xvars]
    X = sm.add_constant(X)

    mod = sm.OLS(y, X)
    
    assert not (robust and cluster) #need to pick one of robust or cluster
    
    if robust:
        return mod.fit(cov_type='HC1')
    
    if cluster:
        groups = regdf[cluster].unique().tolist()
        group_ints = regdf[cluster].apply(lambda x: groups.index(x))
        return mod.fit(cov_type='cluster', cov_kwds={'groups': group_ints})
    
    return mod.fit()


def reg_for_each_combo(df, yvar, xvars, **reg_kwargs):
    """
    Takes each possible combination of xvars (starting from each var individually, then each pair
    of vars, etc. all the way up to all xvars), and regresses yvar on each set of xvars. Returns
    a list of fitted regressions.
    """
    reg_list = []
    for i in range(1, len(xvars) + 1):
        for combo in itertools.combinations(xvars, i):
            x = list(combo)
            reg_list.append(reg(df, yvar, x, **reg_kwargs))
            
    return reg_list

def reg_for_each_xvar_set(df, yvar, xvars_list, **reg_kwargs):
    """
    Runs regressions on the same y variable for each set of x variables passed. xvars_list 
    should be a list of lists, where each individual list is one set of x variables for one model.
    Returns a list of fitted regressions.
    """
    return [reg(df, yvar, x, **reg_kwargs) for x in xvars_list]

def select_models(reg_list, keepnum, xvars):
    """
    Takes a list of fitted regression models and selects among them based on adjusted R-Squared. For each
    number of variables involved in the regressions, keepnum with the highest R-squareds will be kept.
    
    For example, if reg_list contains 3 regressions with two variables and 6 regressions with three variables,
    and keepnum is 2, will return a list of four regressions, 2 with two variables and 2 with three variables.
    """
    outlist = []
    for i in range(1, len(xvars) + 1):
        reg_list_match = [reg for reg in reg_list if reg.df_model == i] #select models with this many variables
        try:
            r2_min = sorted([reg.rsquared_adj for reg in reg_list_match])[-keepnum] #gets keepnumth highest r2
        except IndexError: #should happen once there are less models run than keepnum (i.e. with all xvars)
            r2_min = sorted([reg.rsquared_adj for reg in reg_list_match])[0] #gets lowest r2 (keep all)
        outlist += [reg for reg in reg_list_match if reg.rsquared_adj >= r2_min]
    return outlist

def produce_summary(reg_list, stderr=False, float_format='%0.1f'):

    summ =  summary_col(reg_list, stars=True, float_format=float_format,
            info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                      'R2':lambda x: "{:.2f}".format(x.rsquared),
                      'Adj-R2':lambda x: "{:.2f}".format(x.rsquared_adj)})
    
    if not stderr:
        summ.tables[0].drop('', axis=0, inplace=True) #drops the rows containing standard errors
        
    return summ

def reg_for_each_xvar_set_and_produce_summary(df, yvar, xvars_list, robust=True, 
                                              cluster=False, stderr=False, float_format='%0.1f'):
    """
    Convenience function to run regressions for every set of xvars passed
    and present them in a summary format. Returns a tuple of (reg_list, summary) where reg_list
    is a list of fitted regression models, and summary is a single dataframe of results.
    
    Required inputs:
    df: pandas dataframe containing regression data
    yvar: str, column name of y variable
    xvars_list: list of lists of strs, each individual list has column names of x variables for that model
    
    Optional inputs:
    robust: bool, set to True to use heterskedasticity-robust standard errors
    cluster: False or str, set to a column name to calculate standard errors within clusters
             given by unique values of given column name
    stderr: bool, set to True to keep rows for standard errors below coefficient estimates
    
    Note: only specify at most one of robust and cluster.
    """
    reg_list = reg_for_each_xvar_set(df, yvar, xvars_list, robust=robust, cluster=cluster)
    summ = produce_summary(reg_list, stderr=stderr, float_format=float_format)
    return reg_list, summ

def reg_for_each_combo_select_and_produce_summary(df, yvar, xvars, robust=True, cluster=False,
                                                  keepnum=5, stderr=False, float_format='%0.1f'):
    """
    Convenience function to run regressions for every combination of xvars, select the best models,
    and present them in a summary format. Returns a tuple of (reg_list, summary) where reg_list
    is a list of fitted regression models, and summary is a single dataframe of results
    
    Required inputs:
    df: pandas dataframe containing regression data
    yvar: str, column name of y variable
    xvars: list of strs, column names of all possible x variables
    
    Optional inputs:
    robust: bool, set to True to use heterskedasticity-robust standard errors
    cluster: False or str, set to a column name to calculate standard errors within clusters
             given by unique values of given column name
    keepnum: int, number to keep for each amount of x variables. The total number of outputted
             regressions will be roughly keepnum * len(xvars)
    stderr: bool, set to True to keep rows for standard errors below coefficient estimates
    
    Note: only specify at most one of robust and cluster.
    
    """
    reg_list = reg_for_each_combo(df, yvar, xvars, robust=robust, cluster=cluster)
    outlist = select_models(reg_list, keepnum, xvars)
    summ = produce_summary(outlist, stderr=stderr, float_format=float_format) 
    return outlist, summ

Overwriting reg.py


# Summarize

In [11]:
%%writefile summ.py

import numpy as np

def summary_stats(df, pct_vars=None, int_vars=None, float_vars=None, count=False):
    """
    Generates a transposed df.describe() table where pct_vars are formatted with two
    decimal percentages, int_vars are formatted with zero decimal places, and float_
    vars are formatted with two decimal places.
    
    """
    stats = ['mean', 'std', 'min', '25%','50%','75%','max']
    all_vars = _if_lists_exist_then_combine(pct_vars, int_vars, float_vars)
    summ = df[all_vars].replace([np.inf, -np.inf], np.nan).dropna().describe().T
    apply_formatting(summ, stats, pct_vars=pct_vars, int_vars=int_vars, float_vars=float_vars)
    
#     formats = _if_vars_list_exists_then_add_to_format_dict(pct_vars, int_vars, float_vars)
    
#     _apply_formatting_from_format_dict(summ, formats, stats)

    if count:
        #Format count column individually
        summ['count'] = summ['count'].apply(lambda x: '{0:.0f}'.format(x))
    else:
        summ.drop('count', axis=1, inplace=True)
        
    return summ

def summ_vars_single_stat_by_groups(df, groupvar, pct_vars=None, int_vars=None, float_vars=None, stat='mean'):
    """
    Generates a table where groups are columns and rows are variables, where the values
    are a single summary stat
    """
    all_vars = _if_lists_exist_then_combine(pct_vars, float_vars, int_vars)

    group = df.groupby(groupvar)[all_vars]
    func = getattr(group, stat) #gets .mean(), .median(), etc.
    summ = func().T #applies .mean(), .median(), etc.
    apply_formatting(summ, summ.columns, pct_vars=pct_vars,
                               float_vars=float_vars, int_vars=int_vars)
    return summ

def apply_formatting(df, cols, pct_vars=None, int_vars=None, float_vars=None):
    """
    Applies percentage, integer, and float formatting to a summary table. Variables must
    be in rows, with statistics in columns.
    
    Note: inplace
    """
    formats = _if_vars_list_exists_then_add_to_format_dict(pct_vars, int_vars, float_vars)
    
    _apply_formatting_from_format_dict(df, formats, cols)

def _if_lists_exist_then_combine(*args):
    out = []
    for a in args:
        if a:
            out += a
    return out

def _if_vars_list_exists_then_add_to_format_dict(pct_vars, int_vars, float_vars):
    formats = {
        0: '{:.2%}',
        1: '{0:.0f}',
        2: '{0:.2f}'
    }
    for i, var_list in enumerate([pct_vars, int_vars, float_vars]):
        if var_list:
            formats[tuple(var_list)] = formats.pop(i) #replaces int key with tuple of vars
    return formats

def _apply_formatting_from_format_dict(df, formats, cols):
    """
    Note: inplace
    """
    for k in formats: #apply appropriate format to stats columns but not count column
        if not isinstance(k, int): #skip empty lists - these are marked by an integer
            df.loc[list(k), cols] = df.loc[list(k), cols].applymap(
                lambda x: formats[k].format(x))

Overwriting summ.py


# WRDS

In [12]:
%%writefile wrds.py

from io import StringIO
import sys
import functools

# class WRDS:
    
#     def __init__(self):
#         import wrds
#         self.wrds_obj = wrds
        
#     def get(self, *args, **kwargs):
#         """
#         See docstring of dero.wrds.get
#         """
#         return get(*args, wrds_obj=self.wrds_obj, **kwargs)
    
#     def sql(self, *args, **kwargs):
#         """
#         See docstring of dero.wrds.sql
#         """
#         return sql(*args, wrds_obj=self.wrds_obj, **kwargs)

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        sys.stdout = self._stdout


def login_if_needed(func):
    """
    This decorator is to be used after importing wrds. Will log in again
    if connection is timed out.
    
    Usage:
    
    import wrds
    
    @login_if_needed
    def get_msi():
        return wrds.sql('select * from CRSP.MSI')
    """

    @functools.wraps(func)
    def func_or_login_and_func(*args,**kwargs):
        import wrds
        kwargs.update({'wrds_obj':wrds})
        with Capturing() as output:
            result = func(*args, **kwargs)
        if any(['Connection reset by peer' in s for s in output]): #connection error
            wrds.CONN = wrds._authenticate(wrds.username, wrds.password)
            return func(*args, **kwargs)
        return result

    return func_or_login_and_func

def get(libname, tablename, getvars=False, where=False, subset=False, distinct=False):
    """
    Executes a standard SQL query to get variables from table. Passing an int to subset
    causes only that many rows to be returned.
    
    Required inputs: (note: none of these are case sensitive)
    libname: str, name of library in wrds
    tablename: str, name of table within library in wrds
    
    Optional inputs:
    getvars: False, str, or list of strs, names of columns to pull from table. If False will
             pull all columns.
    where: False or str, SQL where expression without the word where.
            Examples:
                'permno = 10516 and askhi > 1000'
                'date = "04jan2013"d'
                'date between "07jan2013"d and "08jan2013"d'
    subset: False or int, set to an int to keep that many observations
    distinct: bool, set to true to select distinct
    """
#     if not wrds_obj:
#         import wrds
#         wrds_obj = wrds
    
    if isinstance(getvars, str):
        getvars = [getvars]
    
    select = 'select ' + ('distinct ' if distinct else '')
    
    if getvars:
        var_string =  ', '.join([g.lower() for g in getvars])
    else:
        var_string = '*'
        
    table_string = '.'.join([n.strip().upper() for n in [libname, tablename]])
    query = select + var_string + ' from ' + table_string
    
    if subset:
        query += ' (obs={})'.format(subset)
        
    if where:
        query += ' where ' + where
    
    return sql(query)

@login_if_needed
def sql(*args, **kwargs):
    """
    Replicates wrds.sql with auto-login if necessary. Pass query string and index. Documentation
    from wrds.sql below:
    
    Run a SQL Query on the WRDS Database.
    data = wrds.sql('select * from CRSP.MSI', 'DATE')

    The second argument gives the column name of the index, if you'd like
    your DataFrame to be indexed.
    """
#     if not wrds_obj:
#         import wrds
#         wrds_obj = wrds

    wrds_obj = kwargs.pop('wrds_obj')
    
    return wrds_obj.sql(*args, **kwargs)

def strip_str(x):
     return x.strip() if isinstance(x, str) else x


def tolist_and_strip_outlist(func):
    """
    This decorator works for functions that return a pandas Series. It converts the series to
    a list and strips white space from the output.
    
    """
    @functools.wraps(func)
    def tolist_and_strip(*args,**kwargs):
        result = func(*args, **kwargs)
        return [r.strip() for r in result.tolist()]

    return tolist_and_strip

@tolist_and_strip_outlist
def all_libraries():
    """
    Returns every library in WRDS
    """
    return sql('select distinct libname from dictionary.tables')['libname']


@tolist_and_strip_outlist
def all_tables_in_library(libname):
    """
    Returns every table in a library in WRDS
    """
    return sql('select distinct memname from dictionary.columns ' 
                    'where libname="{}"'.format(libname.upper()))['memname']

@tolist_and_strip_outlist
def _all_columns_in_table(libname, tablename):
    """
    Returns every column in a table in WRDS
    """
    return sql('select name from dictionary.columns ' 
                    'where libname="{}" and memname="{}"'.format(
                        libname.upper(), tablename.upper()))['name']


@tolist_and_strip_outlist
def _all_labels_in_table(libname, tablename):
    """
    Returns every column in a table in WRDS
    """
    return sql('select label from dictionary.columns ' 
                    'where libname="{}" and memname="{}"'.format(
                        libname.upper(), tablename.upper()))['label']

def all_columns_in_table(libname, tablename):
    """
    Args are libname, tablename strings. Returns a dataframe of column names and labels.
    """
    return sql('select name, label from dictionary.columns ' 
                    'where libname="{}" and memname="{}"'.format(
                        libname.upper(), tablename.upper())).applymap(
                            strip_str)
    
def all_tables():
    """
    Returns every table in WRDS
    """
    df = sql('select libname, memname, memlabel, nvar from dictionary.tables '
                  'where not(missing(memlabel))').rename(
                        columns={'nvar':'Number of Columns'})
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

Overwriting wrds.py


# Selenium

In [13]:
%%writefile ext_selenium.py

import selenium, zipfile, os

def apply_proxy_with_authentication_to_chrome_selenium(proxy, port, username, password, directory):
    '''
    Applies a proxy server to Chrome driven by Selenium, with authentication included. This works by
    creating a zip file extension for Chrome which sets the proxy setting and catches prompt
    windows to include username and password info.

    proxy: string, IP or address of your proxy, i.e. 'dubai.wonderproxy.com'
    port: integer, Port for proxy, i.e. 12000
    username: string
    password: string
    directory: string, This should be the wherever you want the zipfile to be placed, i.e. r'C:/Users/John/Desktop'
    '''
    chrome_options = selenium.webdriver.ChromeOptions()
    #self.chrome_options.add_argument('--proxy-server={}'.format(proxy))

    manifest_json = """
    {
        "version": "1.0.0",
        "manifest_version": 2,
        "name": "Chrome Proxy",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking"
        ],
        "background": {
            "scripts": ["background.js"]
        },
        "minimum_chrome_version":"22.0.0"
    }
    """

    background_js = '''
    var config = {{
            mode: "fixed_servers",
            rules: {{
              singleProxy: {{
                scheme: "http",
                host: "{0}",
                port: parseInt({1})
              }},
              bypassList: ["foobar.com"]
            }}
          }};

    chrome.proxy.settings.set({{value: config, scope: "regular"}}, function() {{}});

    function callbackFn(details) {{
        return {{
            authCredentials: {{
                username: "{2}",
                password: "{3}"
            }}
        }};
    }}

    chrome.webRequest.onAuthRequired.addListener(
                callbackFn,
                {{urls: ["<all_urls>"]}},
                ['blocking']
    );

    '''.format(proxy,port,username,password)

    plugin_file = os.path.join(directory,'proxy_auth_plugin.zip')

    with zipfile.ZipFile(plugin_file, 'w') as zp:
        zp.writestr("manifest.json", manifest_json)
        zp.writestr("background.js", background_js)

    chrome_options.add_extension(plugin_file)

    return chrome_options

class AnyEC:
    """ Use with WebDriverWait to combine expected_conditions
        in an OR.
        
    Then call it like...

    from selenium.webdriver.support import expected_conditions as EC
    # ...
    WebDriverWait(driver, 10).until( AnyEC(
        EC.presence_of_element_located(
             (By.CSS_SELECTOR, "div.some_result")),
        EC.presence_of_element_located(
             (By.CSS_SELECTOR, "div.no_result")) ))
        
    """
    def __init__(self, *args):
        self.ecs = args
    def __call__(self, driver):
        for fn in self.ecs:
            try:
                if fn(driver): return True
            except:
                pass

Overwriting ext_selenium.py


# Sympy

In [14]:
%%writefile ext_sympy.py

import datetime, os, sys
import multiprocessing as mp
from sympy import solve

class EquationSolver:
    
    def __init__(self, log_dir=None, debug=False):
        self.log_dir = log_dir
        self.debug = debug
        
        if self.log_dir:
            self.log_list = []
        
    def create_log_file(self):
        process = mp.current_process()._identity
        process_num = str(process[0])
        #name = 'log_' + str(datetime.datetime.now().replace(microsecond=0)).replace(':','.') + '_' + process_num + '.txt'
        name = 'log_' + process_num + '.txt'
        #name = 'log_' + str(datetime.datetime.now().replace(microsecond=0)).replace(':','.') + '.txt'
        if not os.path.exists(self.log_dir): os.makedirs(self.log_dir)
        self.log_path = os.path.join(self.log_dir, name)
                    
        if not os.path.exists(self.log_path):
            with open(self.log_path, 'w') as f:
                f.write('\n')
    
    def log(self, message, error=False, neverprint=False):       
        self.create_log_file()
        
        if error:
            message = 'ERROR: ' + message
        if message != '\n':
            time = datetime.datetime.now().replace(microsecond=0)
            message = str(time) + ': ' + message
        if self.debug and not neverprint:
            sys.stdout.write(message + '\n')
            sys.stdout.flush() #forces output now
        try:
            with open(self.log_path, 'a') as f:
                [f.write(item) for item in self.log_list] #log anything saved in memory that couldn't be written before
                f.write(message)
                f.write('\n')
            self.log_list = []
        except PermissionError: #if someone happened to write to the file at the same time
            self.log_list.append(message) #save it to log later
            self.log_list.append('\n')
            

    def solve_equations(self, selected_tuple, equations=None, params=None):
        """
        Solves a subset of a set of equations. For use with parallel_loop_with_timeout. Combining the two gets
        all possible solutions for an overidentified system using parallel processing. Doing this without the
        timeout can cause sympy to get stuck on some sets of equations.

        Arguments:
        selected_tuple: tuple, indices of the equations to select. Typically created by itertools.combinations.
        equations: full list of equations to solve. The subset of equations will be pulled from this list.
        params: parameters to solve the system in terms of. 

        Typical usage:
        import sympy
        sympy.init_session()
        eqs = []
        eqs.append(Eq(3*x + y, 2))
        eqs.append(Eq(2*x + y, 5))
        eqs.append(Eq(4*x + 2*y, 6))
        params = [x, y]
        
        log_dir ='C:\\Users\\derobertisna.UFAD\\Dropbox\\UF\\Nimal\\V PIN\\Modeling\\Testing\\Equations'
        
        combinations = list(itertools.combinations(range(len(equations)), len(params)))
        solve = dero.EquationSolver(log_dir=log_dir)
        solutions = dero.parallel_loop_with_timeout(solve.solve_equations, combinations, timeout=120,
                                                         equations=eqs, params=params)
        print([sol for sol in solutions if sol[1] not in ([],'timeout')])

        """
        assert equations != None
        assert params != None

        solve_eqs = [equations[i] for i in selected_tuple]

        if self.log_dir: #if we are logging
            self.log('Solving equation set {}:'.format(selected_tuple))
            [self.log(str(eq)) for eq in solve_eqs]

        result = solve(solve_eqs, params)

        if self.log_dir: #if we are logging
            self.log('Result for set {}:'.format(selected_tuple))
            self.log(str(result))

        return result

Overwriting ext_sympy.py


# Multiprocessing

In [15]:
%%writefile ext_multiprocessing.py

import multiprocessing
from functools import partial
import time, timeit, sys
from multiprocessing.dummy import Pool as ThreadPool

from .ext_time import estimate_time

def abortable_worker(func, *args, **kwargs):
    """
    For use with parallel_loop_with_timeout. 
    """
    timeout = kwargs.get('timeout', None)
    del kwargs['timeout']
    p = ThreadPool(1)
    res = p.apply_async(func, args=args, kwds=kwargs)
    try:
        out = res.get(timeout)  # Wait timeout seconds for func to complete.
        return out
    except multiprocessing.TimeoutError:
        print("Aborting due to timeout")
        p.terminate()
        return 'timeout'

def parallel_loop_with_timeout(target, iterlist, timeout=2, **kwargs):
    """
    Parallelizes a loop while imposing a timeout on any individual iteration of the loop. Returns a list
    of tuples where the first elements are the index of iterlist and the second elements are the results.
    If any iteration times out, will still add the tuple to the list but the second element will be 
    'timeout'. 
    
    IMPORTANT NOTE:
    The target function MUST BE IMPORTED. This will not work if you define the target function in your
    current namespace. This is a multiprocessing restriction. The function can be imported from the same
    module as this paralellizing function.
    
    Arguments:
    target: function, must be imported. Must accept one regular argument, for which the function will
            be evaluated for each element of iterlist. Any additional arguments which do not change with
            each iteration must be passed as keyword arguments.
    iterlist: list, arguments to pass to target function. Will call the target function one time for
              each element of the list.
    timeout: int, maximum time to wait for each function call.
    **kwargs: include any keyword arguments to pass to target function 
    
    """
    counter = []
    
    pool = multiprocessing.Pool()
    abortable_func = partial(abortable_worker, target, timeout=timeout)
    
    start_time = timeit.default_timer()
    num_loops = len(iterlist)
    
    
    results = [(i,
              pool.apply_async(abortable_func, args=(arg,), kwds=kwargs, callback=counter.append))
              for i, arg in enumerate(iterlist)]
    pool.close()
    
    try: 
        while len(counter) < num_loops:
            estimate_time(num_loops, len(counter), start_time)
            time.sleep(0.5)
    except (KeyboardInterrupt, SystemExit):
        pool.terminate()
        pool.join()
        print('Exiting.')
        sys.exit(1)

    pool.join()
    return [(result[0], result[1].get()) for result in results]

Overwriting ext_multiprocessing.py


# Core

In [16]:
%%writefile core.py

import collections
import string, pdb
from unidecode import unidecode

class OrderedSet(collections.MutableSet):

    def __init__(self, iterable=None):
        self.end = end = [] 
        end += [None, end, end]         # sentinel node for doubly linked list
        self.map = {}                   # key --> [key, prev, next]
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:        
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)
    
def return_lines(inpath, outpath, term_list, num_lines=3, encoding='utf-8', debug=False):
    '''
    Takes a file containing text as input. Searches through file for terms in term_list. When a term
    is found, takes that line as well as num_lines surrounding that line, and adds to output file.

    Required options:
    inpath:    string, full filepath of input file
    outpath:   string, full filepath of output file
    term_list: list, terms to search for in input file
    num_lines: int, number of lines to return around each result. Passing 0 will return only the line of
               the found term. Passing 2 would return two lines above, and two below, as well as the original
               line, for a total of five lines.
    '''
    def prepare_for_search(search_list):
        '''
        Strips out punctuation, extra spacing, converts special characters, and converts to lower space. Can be used
        with a string or list.
        '''

        if isinstance(search_list,str):
            searchword_no_punc = "".join(char for char in search_list if char not in string.punctuation)
            searchword_no_punc = unidecode(searchword_no_punc)
            return searchword_no_punc.lower().strip()
        elif isinstance(search_list,list):
            out_list = []
            for searchword in search_list:
                searchword_no_punc = "".join(char for char in searchword if char not in string.punctuation)
                searchword_no_punc = unidecode(searchword_no_punc)
                out_list.append(searchword_no_punc.lower().strip())
            return out_list
        else:
            raise ValueError('must provide string or list of strings')
    
    def file_to_dict(file_path, encoding=encoding):
        '''
        Creates a dictionary from a file where the keys are line numbers and values are line strings
        '''
        with open(file_path, 'r', encoding=encoding) as f:
            line_dict = {}
            for line_num, line in enumerate(f):
                line_dict.update({line_num: line.strip()})
        return line_dict
    
    def add_to_output(line_num, outset, last_line, num_lines=num_lines):
        outset.add(line_num)
        for i in range(num_lines):
            up = line_num + i + 1
            down = line_num - i - 1
            if up <= last_line:
                outset.add(up)
            if down >= 0:
                outset.add(down)
        return outset
    
    if isinstance(term_list,str): term_list = [term_list]
    elif not isinstance(term_list,list): raise ValueError('must provide string or list of strings')
    
    search_list_str = 'Search list: ' + ', '.join(term_list)
    search_list = prepare_for_search(term_list) #strips out punctuation, etc.
    split_search_list = [term.split() for term in search_list] #divides multi-word terms into individual words
    first_word_set = {term_split[0] for term_split in split_search_list} #gets a set of the first words to search for
    infile_dict = file_to_dict(inpath)
    
    #If there are no lines in the file, exit and return zeroes
    if infile_dict == {}:
        if debug:
            print('{} is blank, cannot extract lines.'.format(inpath))
        return [0 for item in term_list]
    
    last_line = max([line for line in infile_dict])
    outset = set() #container for line numbers to be outputted
    full_match_count_list = [0] * len(term_list) #this is a counter for how many full matches, will be returned

    with open(inpath, 'r', encoding=encoding) as f_in:
        word_set = first_word_set
        match_count_list = [0] * len(term_list) #we need to count matches individually by term
        for line_num in infile_dict: 
            line = prepare_for_search(infile_dict[line_num]) #strip punctuation, etc. from search line
            word_list = line.split()
            for word in word_list:
                #Add 1 to the element of the match count list corresponding to a matched term word (addition part),
                #otherwise set match count back to zero (this is the multiplication part)
                match_count_list = [match_count * (term_split[match_count] == word) + (term_split[match_count] == word)
                                    for match_count, term_split in zip(match_count_list,split_search_list)]
                #If we've matched any of the terms completely
                if any([len(term_split) == match_count 
                        for match_count, term_split in zip(match_count_list, split_search_list)]):
                    outset = add_to_output(line_num, outset, last_line)
                    #Add to full match list
                    full_match_count_list = [full_match_count + (len(term_split) == match_count)
                                            for match_count, term_split, full_match_count
                                            in zip(match_count_list, split_search_list, full_match_count_list)]
                    #Set any fully matched back to zero
                    match_count_list = [match_count * (1 - (len(term_split) == match_count)) 
                                        for match_count, term_split in zip(match_count_list, split_search_list)]
                else:
                    word_set = {term_split[match_count] 
                                for match_count, term_split in zip(match_count_list, split_search_list)}
    
    with open(outpath,'w', encoding=encoding) as f_out:
        outlist = sorted(list(outset)) #puts lines in proper order
        f_out.write(search_list_str)
        f_out.write('\n')
        for pos, line_num in enumerate(outlist):
            if pos > 0:
                if outlist[pos - 1] != line_num - 1: f_out.write('\n') #split up sections for each match
            len_num = len(str(line_num)) #get length of line number, so that we can line up each line
            num_space = max(10 - len_num, 0) #set amount of spacing to include to left of line
            f_out.write('{}:'.format(line_num) + ' ' * num_space +  '{}'.format(infile_dict[line_num]))
            f_out.write('\n')
            
    return full_match_count_list

def filter_list(inlist, match=None, nomatch=None, remove=True):
    '''
    Returns outlist, inlist
    Takes inlist and finds items which contain match but do not contain nomatch. If remove=True, removes those
    items from inlist.
    '''
    assert not (match == None and nomatch == None)
    if isinstance(match, str):
        match = [match]
    if isinstance(nomatch, str):
        nomatch = [nomatch]
    if nomatch and match:
        q = [(i, n) for n, i in enumerate(inlist) 
                             if all(map(lambda x: x in i,match)) and all(map(lambda x: x not in i,nomatch))]
    elif nomatch:
        q = [(i, n) for n, i in enumerate(inlist) if all(map(lambda x: x not in i,nomatch))]
    else: #only match
        q = [(i, n) for n, i in enumerate(inlist) if all(map(lambda x: x in i,match))]
    if q != []:
        outlist, outindex = zip(*q) #zip(*) unzips
    else:
        outlist, outindex = ([],[])
    if remove:
            inlist = [item for index, item in enumerate(inlist) if index not in outindex]
    return outlist, inlist

Overwriting core.py


# Latex

In [17]:
%%writefile latex.py

import os, datetime, filecmp, shutil, math
import pandas as pd
from io import StringIO

from .data import replace_missing_csv


def date_time_move_latex(tablename,filepath, folder_name='Tables'):
    r"""
    Takes a LaTeX tex and PDF after the PDF's been created by pdflatex and moves it into a table
    folder with the date, checking if the table was just previously created. If it's the same as
    before, it just deletes the file.

    Required arguments:
        tablename: operating system name of the table, without extensions
        filepath: full filepath of table, without table name. put r before quotes as follows: r'C:\Users\Folder'

    """
    def exit_sequence(tablename, filepath):
        os.remove(os.path.join(filepath, str(tablename) + '.aux'))
        os.remove(os.path.join(filepath, str(tablename) + '.log'))
        return

    os.chdir(filepath) #sets working directory to current directory of table
    table_pdf = tablename + ".pdf"
    table_tex = tablename + ".tex"
    table_xlsx = tablename + ".xlsx"
    inpath_pdf = os.path.join(filepath,table_pdf)
    inpath_tex = os.path.join(filepath,table_tex)
    inpath_xlsx = os.path.join(filepath,table_xlsx)

    tables_path = os.path.join(filepath, folder_name) #set table directory
    if not os.path.exists(tables_path): #create a general table directory if it doesn't exist
        os.makedirs(tables_path)

    current_date = datetime.datetime.today().timetuple()
    current_time = datetime.datetime.today().timetuple()
    format_time = [str(current_time[0]),str(current_time[1]),str(current_time[2])]
    for i in range(3):
        if current_time[i] < 10:
            format_time[i] = "0" + str(current_time[i])
    datetime_str = "{}-{}-{}_".format(format_time[0],format_time[1],format_time[2])

    count = 0 #set up count variable
    while True: #continuous loop
        count += 1
        str_count = "Num" + str(count)
        name_str = datetime_str + str_count
        folder_path = os.path.join(tables_path,name_str)
        outpath_tex = os.path.join(folder_path,table_tex)
        outpath_pdf = os.path.join(folder_path,table_pdf)
        outpath_xlsx = os.path.join(folder_path,table_xlsx)
        if os.path.exists(folder_path): #if the folder already exists
            if os.path.exists(outpath_tex): #if there is already a tex file with the same name
                if filecmp.cmp(outpath_tex,inpath_tex) == True: #if this is the same exact table
                    exit_sequence(tablename,filepath)
                    os.remove(inpath_pdf)
                    os.remove(inpath_tex)
                    if os.path.isfile(inpath_xlsx): #if there is an XLSX file, delete it as well
                        os.remove(inpath_xlsx)
                    return #stop
                else: #if there is a tex file with the same name but it's not the same table
                    continue #go to next iteration of loop (change output number)
            else:
                shutil.move(inpath_pdf,outpath_pdf) #moves file
                shutil.move(inpath_tex,outpath_tex) #moves file
                if os.path.isfile(inpath_xlsx): #if Excel file exists, move it
                    shutil.move(inpath_xlsx,outpath_xlsx)
                exit_sequence(tablename,filepath)
                return
        else: #if the folder doesn't exist
            os.mkdir(folder_path) #create the folder
            shutil.move(inpath_pdf,outpath_pdf) #moves file
            shutil.move(inpath_tex,outpath_tex) #moves file
            if os.path.isfile(inpath_xlsx): #if Excel file exists, move it
                    print(inpath_xlsx)
                    print(outpath_xlsx)
                    shutil.move(inpath_xlsx,outpath_xlsx)
            exit_sequence(tablename,filepath)
            return
        
def csv_to_raw_latex(infile, csvstring=False, missing_rep=" - ", formatstr='{:.3f}', skipfix=None):
    '''
    Takes a CSV text file and converts it to a LaTeX formatted list, with each line of the LaTeX
    file as an item in the list.
    
    Required options:
        infile: Full file path of CSV (include r before quotes)
    
    Optional options:
        csvstring: True to pass a CSV string to infile rather than load from file
        missing_rep: Representation for missing numbers, default " - "
        formatstr: Python string for number formatting, for example '{:.3f}' with quotes
        skipfix: String or list of strings of fixes to skip, options are ['&','%','_']
    '''
    latex_list = []
    if not csvstring:
        f = open(infile,'r')
    else:
        from io import StringIO
        f = StringIO(infile)
        
    if skipfix:
        if isinstance(skipfix, str):
            skipfix = [skipfix]
        assert isinstance(skipfix, list)
    
    csv_list = f.readlines()
    miss_csv_list = replace_missing_csv(csv_list,missing_rep)
    latex_list = []
    for i, line in enumerate(miss_csv_list):
        line_string = ''
        for j, item in enumerate(line):
            if j is not 0: #don't put an & before the first item in line
                line_string += ' & '
            #LaTeX character fixes
            if skipfix:
                if '&' not in skipfix:
                    item = item.replace('&', '\&')
                if '%' not in skipfix:
                    item = item.replace('%', '\%')
                if '_' not in skipfix:
                    item = item.replace('_', '\_')
            else: #make all replacements
                item = item.replace('&','\&')
                item = item.replace('%','\%')
                item = item.replace('_','\_')
            if item.find('.') is not -1: #if we are dealing with a number with decimals
                try:
                    item = formatstr.format(float(item))
                except:
                    pass
            item = item.replace('\n','')
            line_string += item
        line_string += " \\\ \n"
        if i is 0: #on the first line, remove quotes from names
            line_string = line_string.replace('''"''','') #strip out quotes
        latex_list.append(line_string)
        
    if not csvstring:
        f.close()
        
    return latex_list

def df_to_pdf_and_move(dflist, outfolder, outname='table', tabular_string='', string_format='{:.3f}', 
                       above_text='', below_text='',
                     font_size=12, caption='', parse_dates=False, missing_rep=' - ', panel_names=None, colname_flags=None,
                      outmethod='pandas'):
    '''
    Takes a dataframe or list of dataframes as input and outputs to a LaTeX formatted table with multiple panels,
    creates a PDF, and moves the LaTeX file and PDF to a dated folder.
    
    Required options:
        dflist:         Dataframe or list of dataframes.
        outfolder:      Output folder for LaTeX file and PDF. Inside of this folder, a folder called Tables will be created,
                        inside of which the two files will be put inside another folder with the date.
        
        
    Optional options:
        outname:        Name of output table, default is table
        tabular_string: Can take any string that would normally used in tabular (i.e. rrr for three columns right aligned 
                        as well as L{<width>), C{<width>}, and R{<width>} (i.e. L{3cm}) for left, center, and right aligned
                        fixed width. Additionally . aligns on the decimal. Default is first column left aligned, rest 
                        center aligned.
        string_format:  String or list of format of numbers in the table. Please see Python number formats. {:.3f} is 
                        three decimals, the default.
        font_size:      Font size, default 12
        caption:        Title of table
        missing_rep:    Representation for missing numbers, default " - "
        panel_names:    Python list of names of each panel, to go below column names, e.g. ['Table','Other Table']
        colname_flags:  Python list of yes or no flags for whether to display column names for each panel. Default is to
                        display column names only for the first panel, as usually the panels have the same columns. 
                        The default input for a three panel table would be ['y','n','n']
        outmethod:      String, 'pandas' or 'csv'. If 'pandas', uses pandas' built in df.to_latex() to build latex. If
                        'csv', uses df.to_csv() and then dero.raw_csv_to_latex(). The latter case is useful when the table
                        itself contains latex expressions.
    
    '''   
    if isinstance(dflist, pd.DataFrame):
        dflist = [dflist]
    assert isinstance(dflist, list)
    if isinstance(string_format, str):
        string_format = [string_format] * len(dflist)
    assert isinstance(string_format, list)
    
    def is_number(s):
        try:
            float(s)
            return True
        except (ValueError, TypeError):
            return False

    outname_tex = str(outname) + ".tex"
    outname_pdf = str(outname) + ".pdf"
    outpath = os.path.join(outfolder, outname_tex)
    latex_string_list = [] #set container for final LaTeX table contents
    if (colname_flags is None) or (len(colname_flags) is not len(dflist)): #if the user didn't specify whether to use colnames, or they specified an incorrect number of flags
        colname_flags = ['y'] #set first colnames to show
        for i in range(len(dflist) - 1):
            colname_flags.append('n') #set rest of colnames not to show
    panel_order = -1
    for i, df in enumerate(dflist): #for each csv in the list
        df = dflist[i].applymap(lambda x: string_format[i].format(float(x)) if is_number(x) else x)
        df = df.fillna(missing_rep)
        if outmethod.lower() == 'pandas':
            latex_list = [line for line in df.to_latex().split('\n') if not line.startswith('\\')]
        elif outmethod.lower() == 'csv':
            latex_list = [line for line in csv_to_raw_latex(df.to_csv(), missing_rep=missing_rep,
                                                        csvstring=True, skipfix='_') if not line.startswith('\\')]
        number_of_columns = 1 + latex_list[0].count(' & ') #number of columns is 1 + number of seperators
        if panel_names is not None and panel_names[i]:
            panel_order += 1 #In combination with next line, sets panel to A, etc.
            panel_letter = chr(panel_order + ord('A')) #sets first panel to A, second to B, and so on
            #LaTeX formatting code
            latex_list.insert(1,r'\midrule \\[-11pt]')
            latex_list.insert(2,'\n')
            latex_list.insert(3,r'\multicolumn{' + str(number_of_columns) + '}{c}{Panel '+ panel_letter + ': ' + panel_names[i] + '} \\\ \\\[-11pt]')
            latex_list.insert(4,'\n')
            latex_list.insert(5,r'\midrule')
            latex_list.insert(6,'\n')
        else: #if there is no panel name, just put in a midrule
            latex_list.insert(1,r'\midrule')
            latex_list.insert(2,'\n')
        if colname_flags[i].lower() in ('n','no'): #if the flag for colnames is no for this panel
            latex_list = latex_list[1:] #chop off colnames
        latex_string = "\n".join(latex_list) #convert list to string
        latex_string_list.append(latex_string) #add this csv's LaTeX table string to the full list of LaTeX table strings


    if tabular_string == "": #set default tabular format
        tabular_string = 'l' + 'c' * (number_of_columns - 1) #first column left aligned, rest centered
    
    #Set list of lines to be written to output file at beginning
    latex_header_list = [r'\documentclass[' + str(font_size) + 'pt]{article}',r'\usepackage{amsmath}',r'\usepackage{pdflscape}',r'\usepackage[margin=0.3in]{geometry}',
                         r'\usepackage{dcolumn}',r'\usepackage{booktabs}',r'\usepackage{array}', r'\usepackage{threeparttable}',
                         r'\newcolumntype{L}[1]{>{\raggedright\let\newline\\\arraybackslash\hspace{0pt}}m{#1}}',
                         r'\newcolumntype{C}[1]{>{\centering\let\newline\\\arraybackslash\hspace{0pt}}m{#1}}',
                         r'\newcolumntype{R}[1]{>{\raggedleft\let\newline\\\arraybackslash\hspace{0pt}}m{#1}}',
                         r'\newcolumntype{.}{D{.}{.}{-1}}',r'\title{\LaTeX}',r'\date{}',r'\author{Nick DeRobertis}',
                         r'\begin{document}',r'\begin{table}',r'\centering',r'\begin{threeparttable}',
                         above_text,r'\caption{' + caption + '}',r'\begin{tabular}{' + tabular_string + '}',
                         r'\toprule']

    #Set list of lines to be written to output file at end
    latex_footer_list = [r'\bottomrule',r'\end{tabular}',r'\begin{tablenotes}[para,flushleft]',r'\item ' + below_text,r'\end{tablenotes}',r'\end{threeparttable}',r'\end{table}',r'\end{document}']

    #Actually write to file
    with open(outpath,'w') as f:
        for line in latex_header_list: #write each line in the header list, with carriage returns in between
            f.write(line)
            f.write("\n")
        for latex_string in latex_string_list: #write each csv table to file in LaTeX format
            f.write(latex_string)
        for line in latex_footer_list: #write each line in the footer list, with carriage returns in between
            f.write(line)
            f.write("\n")
        f.close()


    os.chdir(outfolder) #changes working filepath
    os.system('pdflatex ' + '"' + outname_tex + '"') #create PDF
    date_time_move_latex(outname,outfolder) #move table into appropriate date/number folder
    
def latex_equations_to_pdf(latex_list, directory, name='Equations', below_text=None,
                           math_size=18, text_size=14, title=None, para_space='1em',
                          inline=False):
    script_size = math.ceil(math_size * (2/3))
    scriptscript_size = math.ceil(math_size * .5)
    assert text_size in (8, 9, 10, 11, 12, 14, 17, 20) #latex allowed font sizes
    
    if inline:
        surround_char_beg = '$'
        surround_char_end = '$'
    else:
        surround_char_beg = r'\begin{dmath}'
        surround_char_end = r'\end{dmath}'

    
    headers = [r'\documentclass[{} pt]{{extarticle}}'.format(text_size), 
               #First size is text size, second is math size, third is script size,
               #fourth is scriptscript size
               r'\DeclareMathSizes{{{0}}}{{{1}}}{{{2}}}{{{3}}}'.format(
                    text_size, math_size, script_size, scriptscript_size),
               r'\usepackage{amsmath}',
               r'\usepackage{breqn}',
               r'\usepackage[margin=0.3in]{geometry}',
              r'\author{Nick DeRobertis}' ,r'\begin{document}', r'\setlength{{\parskip}}{{{}}}'.format(para_space)]
    footers = [r'\end{document}']
    name_tex = name + '.tex'
    file_path = os.path.join(directory, name_tex)
    with open(file_path, 'w') as f:
        f.write('\n'.join(headers) + '\n')
        [f.write(surround_char_beg + '{}'.format(line) + surround_char_end + '\n\n') for line in latex_list]
        if below_text:
            f.write('\n' + below_text + '\n')
        f.write('\n'.join(footers))
        
    os.chdir(directory)
    os.system('pdflatex ' + '"' + name_tex + '"') #create pdf
    date_time_move_latex(name, directory, 'Equations')

Overwriting latex.py


# Math

In [18]:
%%writefile ext_math.py

import numpy as np
import pandas as pd
from collections import Counter


def transition_matrix(states):
    """
    Creates a numpy array containing the probability of transition from one state to another. Must 
    pass states as a list of lists, where each inner list is an observation of state changes.
    
    For example:
    states = [
    [2,1,3,1,2,3,1],
    [1,2,2,2]
    ]
    
    Produces:
    array([[ 0.        ,  0.66666667,  0.33333333],
           [ 0.25      ,  0.5       ,  0.25      ],
           [ 1.        ,  0.        ,  0.        ]])
    """

    max_state = max([max(l) for l in states]) + 1
    out = np.zeros((max_state,max_state))
    for (x,y), c in Counter(
        [transition for state_list in states for transition in \
         zip(state_list, state_list[1:])]).items():
        out[x,y] = c
        
    return out/out.sum(axis=1)[:,None]

def _map_to_numbers(string_list):
    """
    Returns a tuple of mapping from numbers to strings, then from strings to numbers
    """
    return {i: item for i, item in enumerate(string_list)}, {item: i for i, item in enumerate(string_list)}
    
def transition_dataframe(states):
    """
    Creates a pandas dataframe containing the probability of transition from one state to another. Must 
    pass states as a list of lists, where each inner list is an observation of state changes.
    
    For example:
    states = [
        ['b','a','c','a','b','c','a'],
        ['a','b','b','b']
    ]
    
    Produces:     a             b            c
         a  [ 0.        ,  0.66666667,  0.33333333],
         b  [ 0.25      ,  0.5       ,  0.25      ],
         c  [ 1.        ,  0.        ,  0.        ]
    """
    unique_states = sorted(set([state for state_list in states for state in state_list]))
    #get dicts which map 0, 1, 2, etc. to given states
    number_to_string_map, string_to_number_map = _map_to_numbers(unique_states) 
    #transform strings to mapped numbers
    state_numbers = [[string_to_number_map[state] for state in state_list] for state_list in states]
    out = pd.DataFrame(transition_matrix(state_numbers))
    return out.rename(columns=number_to_string_map, index=number_to_string_map)

Overwriting ext_math.py


# Decorators

In [19]:
%%writefile decorators.py

import functools, sys
from time import sleep
import types

def apply_decorator_to_all_functions_in_module(module, decorator):
    """
    Usage:
    import module
    import dero
    
    def my_decorator():
        ...
    
    dero.decorators.apply_decorator_to_all_functions_in_module(module, my_decorator)
    
    module.whatever_function() #decorator applied
    
    """
    for k,v in vars(module).items():
        if isinstance(v, types.FunctionType):
            setattr(module, k, decorator(v))

def simple_decorator(decorator):
    '''This decorator can be used to turn simple functions
    into well-behaved decorators, so long as the decorators
    are fairly simple. If a decorator expects a function and
    returns a function (no descriptors), and if it doesn't
    modify function attributes or docstring, then it is
    eligible to use this. Simply apply @simple_decorator to
    your decorator and it will automatically preserve the
    docstring and function attributes of functions to which
    it is applied.
    
    Seems to only work for decorators without arguments.'''
    def new_decorator(f):
        g = decorator(f)
        g.__name__ = f.__name__
        g.__doc__ = f.__doc__
        g.__dict__.update(f.__dict__)
        return g
    # Now a few lines needed to make simple_decorator itself
    # be a well-behaved decorator.
    new_decorator.__name__ = decorator.__name__
    new_decorator.__doc__ = decorator.__doc__
    new_decorator.__dict__.update(decorator.__dict__)
    return new_decorator

@simple_decorator
def dump_args(func):
    "This decorator dumps out the arguments passed to a function before calling it"
    argnames = func.__code__.co_varnames[:func.__code__.co_argcount]
    fname = func.__name__

    def echo_func(*args,**kwargs):
        print(fname, ":", ', '.join(
            '%s=%r' % entry
            for entry in list(zip(argnames,args)) + list(kwargs.items())))
        return func(*args, **kwargs)

    return echo_func

def retries(max_tries, delay=1, backoff=2, exceptions=(Exception,), hook=None):
    """Function decorator implementing retrying logic.

    delay: Sleep this many seconds * backoff * try number after failure
    backoff: Multiply delay by this factor after each failure
    exceptions: A tuple of exception classes; default (Exception,)
    hook: A function with the signature myhook(tries_remaining, exception);
          default None

    The decorator will call the function up to max_tries times if it raises
    an exception.

    By default it catches instances of the Exception class and subclasses.
    This will recover after all but the most fatal errors. You may specify a
    custom tuple of exception classes with the 'exceptions' argument; the
    function will only be retried if it raises one of the specified
    exceptions.

    Additionally you may specify a hook function which will be called prior
    to retrying with the number of remaining tries and the exception instance;
    see given example. This is primarily intended to give the opportunity to
    log the failure. Hook is not called after failure if no retries remain.
    """
    def dec(func):
        @functools.wraps(func)
        def f2(*args, **kwargs):
            mydelay = delay
            tries = list(range(max_tries))
            tries.reverse()
            for tries_remaining in tries:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    if tries_remaining > 0:
                        if hook is not None:
                            hook(tries_remaining, e, mydelay)
                        sleep(mydelay)
                        mydelay = mydelay * backoff
                    else:
                        raise
                else:
                    break
        return f2
    return dec

Overwriting decorators.py


# Time

In [20]:
%%writefile ext_time.py

import math, time, datetime, timeit

def time_elapsed(seconds):
    '''
    Takes an amount of time in seconds and converts it into how a human would say it. 
    
    Required Options:
    seconds: time in number of seconds (raw number, int or float). 
    
    ''' 
    if seconds > 60: #if this is greater than a minute
        if seconds > 60 * 60: #if this is greater than an hour
            if seconds > 60 * 60 * 24: #if this is greater than a day
                if seconds > 60 * 60 * 24 * 30: #if this is greater than a month (approx.):
                    months = math.trunc(seconds / (60 * 60 * 24 *30))
                    seconds -= months * (60 * 60 * 24 * 30)
                    days = math.trunc(seconds / (60 * 60 * 24))
                    seconds -= days * (60 * 60 * 24)
                    hours = math.trunc(seconds / (60 * 60))
                    seconds -= hours * (60 * 60)
                    minutes = math.trunc(seconds / 60)
                    seconds -= minutes * 60
                    seconds = math.trunc(seconds)
                    time_str = "{} months, {} days, {} hours, {} minutes, {} seconds.".format(
                        months, days, hours, minutes, seconds)
                else:
                    days = math.trunc(seconds / (60 * 60 * 24))
                    seconds -= days * (60 * 60 * 24)
                    hours = math.trunc(seconds / (60 * 60))
                    seconds -= hours * (60 * 60)
                    minutes = math.trunc(seconds / 60)
                    seconds -= minutes * 60
                    seconds = math.trunc(seconds)
                    time_str = "{} days, {} hours, {} minutes, {} seconds.".format(days, hours, minutes, seconds)
            else: #if this is greater than an hour but less than a day
                hours = math.trunc(seconds / (60 * 60))
                seconds -= hours * (60 * 60)
                minutes = math.trunc(seconds / 60)
                seconds -= minutes * 60
                seconds = math.trunc(seconds)
                time_str = "{} hours, {} minutes, {} seconds.".format(hours, minutes, seconds)
        else: #if this is greater than a minute but less than an hour
            minutes = math.trunc(seconds / 60)
            seconds -= minutes * 60
            seconds = math.trunc(seconds)
            time_str = "{} minutes, {} seconds.".format(minutes, seconds)
    else: #if this is less than a minute
        seconds = math.trunc(seconds)
        time_str = "{} seconds.".format(seconds)
        
    return time_str
        
def estimate_time(length,i,start_time,output=True):
    '''
    Returns the estimate of when a looping operation will be finished. 
    
    HOW TO USE:
    This function goes at the end of the loop to be timed. Outside of this function at the beginning of the
    loop, you must start a timer object as follows:
    
    start_time = timeit.default_timer()
    
    So the entire loop will look like this:
    
    my_start_time = timeit.default_timer()
    for i, item in enumerate(my_list):
    
        #Do loop stuff here
        
         estimate_time(len(my_list),i,my_start_time)
         
    REQUIRED OPTIONS:
    length:     total number of iterations for the loop
    i:          iterator for the loop
    start_time: timer object, to be started outside of this function (SEE ABOVE)
    
    OPTIONAL OPTIONS:
    output: specify other than True to suppress printing estimated time. Use this if you want to just store the time
            for some other use or custom output. The syntax then is as follows:
    
    my_start_time = timeit.default_timer()
    for i, item in enumerate(my_list):
        
        #Do loop stuff here
        
        my_timer = estimate_time(len(my_list),i,my_start_time, output=False)
        print("I like my output sentence better! Here's the estimate: {}".format(my_timer))
    
    '''
    avg_time = (timeit.default_timer() - start_time)/(i + 1)
    loops_left = length - (i + 1)
    est_time_remaining = avg_time * loops_left
    est_finish_time = datetime.datetime.now() + datetime.timedelta(0,est_time_remaining)
    
    if output == True:
        print("Estimated finish time: {}. Completed {}/{}, ({:.0%})".format(est_finish_time, i, length, i/length), end="\r")
    
    return est_finish_time

def increment_dates(start_date,end_date,frequency='a'):
    '''
    Returns a list of dates inbetween start and end dates. start_date and end_date should be in one of the following
    date formats:
        mm/dd/yyyy, mm/yyyy, yyyy
    Dates should be frequency should be a single letter, 'a' for annual, 'm' for monthly, 'w' for weekly, and 'd' for daily
    '''
    #Find number of slashes to determine how to parse date
    number_of_slashes = []
    number_of_slashes.append(start_date.count('/'))
    number_of_slashes.append(end_date.count('/'))
    date_formats = ['%Y','%Y'] #set container for date formats
    
    for i, number in enumerate(number_of_slashes):
        if number == 0: #no slashes means interpret as year
            pass #already set as year in container
        if number == 1: #one slash means interpret as month/year
            date_formats[i] = '%m/%Y'
        if number == 2: #one slash means interpret as month/year
            date_formats[i] = '%m/%d/%Y'
    
    start = datetime.datetime.strptime(start_date, date_formats[0]).date()
    end   = datetime.datetime.strptime(end_date,   date_formats[1]).date()
    delta = end - start
    
    number_of_years = end.year - start.year
    number_of_months = end.month - start.month
    number_of_days = end.day - start.day
    
    if frequency == 'd': number_of_periods = delta.days + 1 
    if frequency == 'w': number_of_periods = math.ceil((delta.days + 1)/7) 
    if frequency == 'a': number_of_periods = math.ceil((delta.days + 1)/365)
    if frequency == 'm': number_of_periods = math.ceil((delta.days + 1)/(365/12))
    
    outlist = []
    for period in range(number_of_periods):
        if frequency == 'd': 
            new_date = start + datetime.timedelta(days=period)
            outlist.append(str(new_date.month) + '/' + str(new_date.day) + '/' + str(new_date.year))
        if frequency == 'w': 
            new_date = start + datetime.timedelta(weeks=period)
            outlist.append(str(new_date.month) + '/' + str(new_date.day) + '/' + str(new_date.year))
        if frequency == 'a': outlist.append(start.year + period)
        if frequency == 'm':
            new_period = period - 1
            current_month = (start.month + new_period)
            current_year = start.year
            years_passed = math.floor(current_month/12)
            current_year += years_passed
            current_month -= 12 * years_passed - 1
            if current_month > 12:
                pass
            outlist.append(str(current_month) + '/' + str(current_year))
    
    return outlist

Overwriting ext_time.py


# Data

In [21]:
dero_path = os.path.join(orig_path, 'dero')
os.chdir(dero_path)

In [22]:
%%writefile data.py

from sas7bdat import SAS7BDAT
import pandas as pd
import os, datetime, warnings, sys

from .ext_pandas import expand_time, cumulate, convert_sas_date_to_pandas_date, reg_by, factor_reg_by, load_sas, \
                    long_to_wide, year_month_from_date, join_col_strings
    
from .compustat import convert_gvkey, load_compustat, merge_compustat


def replace_missing_csv(csv_list, missing_rep):
    '''
    Replaces missing items in a CSV with a given missing representation string.
    '''
    full_list = []
    for line in csv_list:
        line_list = line.split(',')
        new_line_list = []
        for item in line_list:
            if item == '': #if the item is missing
                item = missing_rep
            new_line_list.append(item)
        full_list.append(new_line_list)
    return full_list

def merge_dsenames(df, on='TICKER', get='PERMNO', date='Date', 
                   other_byvars=None, 
                   crsp_dir=r'C:\Users\derobertisna.UFAD\Desktop\Data\CRSP'):
    '''
    Merges with dsenames file on on variable (TICKER, PERMNO, PERMCO, NCUSIP, CUSIP6), to get get variable (same list).
    Must have a Date variable in df.
    
    Default is to match on TICKER and pull PERMNO.
    
    Required inputs:
    df: pandas dataframe containing any of (TICKER, PERMNO, PERMCO, NCUSIP, CUSIP6)
    
    Optional inputs:
    on: str, column name to merge on, one of (TICKER, PERMNO, PERMCO, NCUSIP, CUSIP6)
    get: str or list, column or columns to get from dsenames, any of (TICKER, PERMNO, PERMCO, NCUSIP, CUSIP6)
         that aren't already in on
    date: str, column name of date variable
    other_byvars: any other variables signifying groups in the data, prevents from collapsing those groups
    '''
    #Make get a list
    if isinstance(get, str):
        get = [get]
    assert isinstance(get, list)
    
    #Make other byvars a list
    if not other_byvars:
        other_byvars = []
    if isinstance(other_byvars, str):
        other_byvars = [other_byvars]
    assert isinstance(other_byvars, list)
    
    assert on not in get #can't get what we already have
    
    #Pull from CRSP dsenames file
    file = 'dsenames'
    fullpath = os.path.join(crsp_dir, file + '.sas7bdat')
    names_df = load_sas(fullpath)
    
    #Convert NCUSIP to CUSIP6
    if on == 'CUSIP6' or 'CUSIP6' in get:
        names_df['CUSIP6'] = names_df['NCUSIP'].apply(lambda x: x if pd.isnull(x) else x[:6])
    
    names_df['start'] = convert_sas_date_to_pandas_date(names_df['NAMEDT'])
    names_df['end'] = convert_sas_date_to_pandas_date(names_df['NAMEENDT'])
    names_df['end'] = names_df['end'].fillna(datetime.date.today())
    
    
    #Now perform merge
    merged = df[[on, date] + other_byvars].merge(names_df[['start','end', on] + get], how='left', on=on)
    #Drop out observations not in name date range
    valid = (merged[date] >= merged['start']) & (merged[date] <= merged['end'])
    #However if there is not a match, doing merged[valid] would drop the observation instead of leaving nan
    #Therefore, take merged[valid] and merge back again to original
    new_merged = df.merge(merged[valid].drop(['start','end'],axis=1), how='left', on=[on, date] + other_byvars)
    new_merged = new_merged.reset_index(drop=True)
    
    if 'PERMNO' in get:
        #Dsenames has no record of which permno is the primary link when a firm has multiple share classes.
        #To get this information, we must merge ccmxpf_linktable. We want to keep only primary links.
        dups = new_merged[[date, on] + other_byvars].duplicated(keep=False) #series of True or False of whether duplicated row
        if dups.any(): #this means we got more than one permno for a single period/firm/byvars
            duplicated = new_merged[dups].reset_index() #puts index in a column for later use
            not_duplicated = new_merged[~dups]
            
            #Take duplicated, merge to ccmxpf_linktable to get gvkey, and the ones which do not have gvkeys are
            #the non-primary links
            with_gvkey = get_gvkey_or_permno(duplicated, date) #default is to get gvkey with permno
            removed_duplicates = with_gvkey[~pd.isnull(with_gvkey['GVKEY'])].drop('GVKEY', axis=1)
            
            #Set index back
            removed_duplicates.set_index('index', inplace=True)
            
            #Now append back together and sort
            full = not_duplicated.append(removed_duplicates)
            new_merged = full.sort_index()
    
    return new_merged.reset_index(drop=True)

def get_gvkey_or_permno(df, datevar, get='GVKEY', other_byvars=None,
                        crsp_dir=r'C:\Users\derobertisna.UFAD\Desktop\Data\CRSP'):
    """
    Takes a dataframe containing either GVKEY or PERMNO and merges to the CRSP linktable to get the other one.
    """    
    if get == 'GVKEY':
        rename_get = 'gvkey'
        l_on = 'PERMNO'
        r_on = 'lpermno'
    elif get == 'PERMNO':
        rename_get = 'lpermno'
        l_on = 'GVKEY'
        r_on = 'gvkey'
    else:
        raise ValueError('Need get="GVKEY" or "PERMNO"')
        
    #Make other byvars a list
    if not other_byvars:
        other_byvars = []
    if isinstance(other_byvars, str):
        other_byvars = [other_byvars]
    assert isinstance(other_byvars, list)
    
    link_name = 'ccmxpf_linktable.sas7bdat'
    link_path = os.path.join(crsp_dir, link_name)
    
    link = load_sas(link_path)
    link['linkdt'] = convert_sas_date_to_pandas_date(link['linkdt'])
    link['linkenddt'] = convert_sas_date_to_pandas_date(link['linkenddt'])
    #If end date is missing, that means link is still active. Make end date today.
    link['linkenddt'] = link['linkenddt'].fillna(datetime.date.today())
    
    #Remove links with no permno so that they don't match to nans in the input df
    link.dropna(subset=['lpermno'], inplace=True)
    
    merged = df.merge(link[['lpermno','gvkey', 'linkdt', 'linkenddt','linkprim']], how='left', left_on=l_on, right_on=r_on)

    valid = (merged[datevar] >= merged.linkdt) & \
            (merged[datevar] <= merged.linkenddt) & \
            (merged.linkprim == 'P')
        
    merged = merged[valid].drop(['linkdt','linkenddt', 'linkprim', r_on], axis=1).drop_duplicates()
    merged.rename(columns={rename_get:get}, inplace=True)
    
    #Now merge back to the original again to ensure that rows are not deleted
    new_merged = df.merge(merged[['PERMNO','GVKEY', datevar] + other_byvars],
                          how='left', on=[l_on, datevar] + other_byvars)
    
    return new_merged


class GetCRSP:
    
    def __init__(self, debug=False, crsp_dir=r'C:\Users\derobertisna.UFAD\Desktop\Data\CRSP'):
        self.debug = debug
        self.crsp_dir = crsp_dir
        self.loaded_m = False
        self.loaded_d = False
        self.crsp_dfs = {} #container for both monthly and daily crsp df
        
    
    def pull_crsp(self, df, coid='PERMNO', freq='m', get=['PRC','SHROUT'], date='Date', other_byvars=None,
                 time=None, wide=True,
                 abret=False, window=None, cumret=False, includefac=False, includecoef=False, 
                 drop_first=False):
        '''
        Pulls prices and returns from CRSP. Currently supports the monthly file merging on PERMNO.

        WARNING: Will overwrite variables called "Year" and "Month" if merging the monthly file

        coid = string, company identifier (currently supports 'GVKEY', 'TICKER', 'PERMNO', 'PERMCO', 'NCUSIP')
        freq = 'm' for monthly, 'd' for daily
        get = 'PRC', 'RET', 'SHROUT', 'VOL', 'CFACPR', 'CFACSHR', or a list combining any of these
        date = name of datetime column in quotes
        other_byvars = other by vars in dataset besides company identifier and date. Used for long_to_wide
        time = list of integers or None. If not None, will pull the variables for a time difference of the time
               numbers given, and put them in wide format in the output dataset. For instance time=[-12,0,12] with
               freq='m' and get='RET' would pull three returns per observation, one twelve months prior, one contemporaneous,
               and one twelve months later, naming them RET-12, RET0, and RET12. If freq='d' in the same example, it would be
               twelve days prior, etc.
        abret = 1, 3, 4, or False. If 1, 3, or 4 is passed and 'RET' is in get, will calculate abnormal returns according to CAPM,
                3 or 4 factor model, respectively. 
        window = Integer or None. Must provide an integer if abret is not False. This is the number of prior periods to use
                 for estimating the loadings in factor models. For instance, if freq='m', window=36 would use the prior
                 three years of returns to estimate the loadings.
        cumret = 'between', 'zero', 'first', or False. time must not be None and RET in get for this option to matter. 
                 When pulling returns for multiple periods, gives the option to cumulate returns. If False, will just return 
                 returns for the individual periods. 
                 If 'zero', will give returns since the original date. 
                 If 'between', will give returns since the prior requested time period.
                 If 'first', will give returns since the first requested time period.
                 For example, if our input data was for date 1/5/2006, and in the CRSP table we had:
                     permno  date       RET
                     10516   1/5/2006   10%
                     10516   1/6/2006   20%
                     10516   1/7/2006    5%
                     10516   1/8/2006   30%
                 Then get_crsp(df, time=[1,3], get='RET', cumret=None) would return:
                     permno  date       RET1  RET3
                     10516   1/5/2006   20%   30%
                 Then get_crsp(df, time=[1,3], get='RET', cumret='between') would return:
                     permno  date       RET1  RET3
                     10516   1/5/2006   20%   36.5%
                 Then get_crsp(df, time=[1,3], get='RET', cumret='zero') would return:
                     permno  date       RET1  RET3
                     10516   1/5/2006   20%   63.8%
                 The output for cumret='first' would be the same as for cumret='zero' because the first period is period zero.
                 Had time been =[-1, 1, 3], then returns would be calculated from period -1 to period 1, and period -1 to period 3. 
        includefac = Boolean, True to include factors in output
        includecoef = Boolean, True to include factor coefficients in output
        wide = True for output data to be wide form, False for long form. Only applies when time is not None.
        drop_first = bool, set to True to drop observations for first time. Can only be used when time != None, and
                     when cumret != False. This is a
                     convenience function for estimating cumulative return windows. For example, if time = [-1, 1], 
                     then the typical output would include both cum_RET-1 and cum_RET1. All we actually care about is the
                     cumulative return over the window, which is equal to cum_RET1. drop_first=True will drop out 
                     RET-1 and cum_RET-1 from the output.
        debug: bool, set to True to restrict CRSP to only PERMNOs 10516 (gvkey=001722) and 10517 (gvkey=001076)

        Typical usage:
        Calculating a return window with abnormal returns included:
            get_crsp(df, get='RET', freq='d', time=[-1, 1], cumret='between', abret=4, window=250, drop_first=True)
        '''  
        self.df = df.copy()
        self.coid = coid
        self.freq = freq
        self.get = get
        self.date = date
        self.other_byvars = other_byvars
        self.time = time
        self.wide = wide
        self.abret = abret
        self.window = window
        self.cumret = cumret
        self.includefac = includefac
        self.includecoef = includecoef
        self.drop_first = drop_first
        
        self._log('Initializing pull_crsp function')
    
        self._clear_old_variables()
        self._check_inputs()
        self._load_crsp()
        
        if self.time == None and self.abret == False: 
            return self._merge_crsp(self.df, self.date) #if we're not doing anything special, just merge CRSP
        
        if self.time:
            self._handle_time()
        else:
            self._handle_no_time()
            
        self._create_key()
        
        if self.abret:
            self._handle_abret()
        else:
            self.long_df = self._merge_crsp(self.long_df, date='Shift Date') #time but not abret        
        
        if self.cumret:
            self._handle_cumret()
        
        self.long_df.drop('key', axis=1, inplace=True)
        
        if self.wide:
            return self._long_to_wide()
        else:
            return self._output_long()
    
    def _clear_old_variables(self):
        """
        If pull_crsp is run multiple times, values of variables would still be set
        """
        try: del self.long_df
        except AttributeError: #long_df not set yet
            pass
        
        
    
    def _output_long(self):
        if self.debug:
            return self.long_df
        for var in ['Shift','Shift Date']:
            try:
                self.long_df.drop(var, axis=1, inplace=True)
            except ValueError:
                pass
        return self.long_df
    
    def _long_to_wide(self):
        self._log('Reshaping long to wide.')
        byvars = [self.coid, self.date]
        if self.other_byvars: #add other byvars for correct long_to_wide
            byvars += self.other_byvars
#         if self.freq == 'm':
#             self.long_df.drop(['Year','Month'], axis=1, inplace=True)
        try:
            self.long_df.drop('Shift Date', axis=1, inplace=True)
        except ValueError: #this is the case where time was not shifted
            #Therefore need to create a colindex which is just 0 for the current time period
            self.long_df['Shift'] = 0
        
        widedf = long_to_wide(self.long_df.reset_index(drop=True),
                              groupvars=byvars, values=self.get, colindex='Shift')
        #chop off zeros from contemporaneous
        return widedf.rename(columns={name: name[:-1] for name in widedf.columns \
                                      if name.endswith('0') and name[:name.find('0')] in self.get}) 
    
    def _handle_cumret(self):
        self._log('Cumret detected.')
        cumvars = ['RET']
        if self.abret: cumvars += ['ABRET']
        self.get += ['cum_' + str(c) for c in cumvars] #get will be used in the end for pivot, need to add pivoting variables
        with warnings.catch_warnings(): #cumulate will raise a warning if time is supplied when method is not between
            warnings.simplefilter('ignore') #suppress that warning
            self._log('Cumulating returns with method {} for time {}.'.format(self.cumret, self.time))
            byvars = ['PERMNO', self.date]
            if self.other_byvars:
                byvars += self.other_byvars
                
            self.long_df = cumulate(self.long_df, cumvars, periodvar='Shift Date', method=self.cumret,
                               byvars=byvars, time=self.time, grossify=True)
            
            
            
            ###########TEMP
#             import pdb
#             pdb.set_trace()
            
            
            
            ############
            
        #Now need to remove unneeded periods
        keep_time = self.time
        if self.drop_first:
            keep_time = self.time[1:]
        self.long_df = self.long_df[self.long_df['Shift'].isin(keep_time)]
            
    def _handle_abret(self):
        self._log('Abret detected.')
        assert isinstance(self.abret, int) and not isinstance(self.abret, bool) #True evaluates as int
        assert isinstance(self.window, int)
        
        facs = ['mktrf']
        if self.abret >= 3:
            facs += ['hml','smb']
        if self.abret == 4:
            facs += ['umd']
        if self.abret not in (1,3,4):
            raise ValueError('Currently only supports 1, 3, and 4-factor models (mktrf, hml, smb, umd)')
        
        prior_wind = [i for i in range(0,-self.window - 1,-1)] #e.g. if window = 3, prior_wind = [0, -1, -2, -3]
        self._log('Creating abret window periods.')
        self.long_df = expand_time(self.long_df, datevar=self.newdate, 
                                   freq=self.freq, time=prior_wind, 
                                   newdate='Window Date', shiftvar='Wind')
        self._log('Merging with CRSP to get abret window data as well as regular data.')       
        self.long_df = self._merge_crsp(df=self.long_df, date='Window Date') 
        self._log('Getting Fama-French factors, then running regressions.')
        self.long_df = get_abret(self.long_df, 'key', fulldatevar='Window Date', freq=self.freq,
                                 abret_fac=self.abret, includecoef=self.includecoef,
                                 includefac=self.includefac)
        
        self._log('Dropping unneeded observations (abret window dates).')
        if self.time:
            #keep only observations for shift dates
            self.long_df = self.long_df[self.long_df['Window Date'] == self.long_df['Shift Date']].reset_index(drop=True) 
        else: #didn't get multiple times per period
            #keep only original observations
            self.long_df = self.long_df[self.long_df['Window Date'] == self.long_df[self.date]].reset_index(drop=True)
        self.get += ['ABRET'] #get will be used in the end for pivot, need to add pivoting variables
        
        drop = ['Wind', 'Window Date'] #variables to be dropped
        coefs = ['coef_' + fac for fac in facs]
        if self.includefac: self.get += facs + ['rf']
        if self.includecoef: self.get += coefs + ['const']
        self.long_df.drop(drop, axis=1, inplace=True)
        
    
    def _create_key(self):
        if self.other_byvars:
            self.byvars += self.other_byvars
        self.long_df['key'] = join_col_strings(self.long_df,self.byvars)
    
    def _handle_time(self):
        self._log('Time detected.')
        #Next section only runs if time is not None
        #Ensure time is of the right type
        if isinstance(self.time, int): self.time = [self.time]
        assert (isinstance(self.time, list) and isinstance(self.time[0], int))
        intermediate_periods = False
        if self.cumret:
            self._log('Cumret detected, will generate intermediate periods.')
            intermediate_periods = True
        self._log('Generating periods {} {}'.format(self.time, '+ itermediate' if intermediate_periods else ''))
        self.long_df = expand_time(self.df, intermediate_periods=intermediate_periods, 
                                   datevar=self.date, freq=self.freq, time=self.time)
        self._log('Finished generating periods. Generating key.')
        self.byvars = ['PERMNO', self.date, 'Shift Date']
        self.newdate = 'Shift Date'
        
    def _handle_no_time(self):
        self.long_df = self.df.copy()
        self._log('Generating key.')
        self.byvars = ['PERMNO', self.date]
        self.newdate = self.date
    
    def _check_inputs(self):
        self._log('Checking inputs.')
        
#         if self.debug:
#             self._log('All inputs: ')
#             self._log(str(self.__dict__))

        #Check get
        if isinstance(self.get, list):
            self.get = [item.upper() for item in self.get]
#             for item in self.get:
#                 assert item in ['PRC','RET','SHROUT','VOL','CFACPR','CFACSHR']
        elif isinstance(self.get, str):
            self.get = self.get.upper()
#             assert self.get in ['PRC','RET','SHROUT','VOL','CFACPR','CFACSHR']
            self.get = [self.get]
        else:
            raise ValueError('''Get should be a list or str containing 'PRC','RET','SHROUT','VOL','CFACPR', or 'CFACSHR'.''')

        #Check to make sure inputs make sense
        assert not ((self.abret == False) and (self.window != None)) #can't specify window without abret
        assert not ((self.abret != False) and (self.window == None)) #must specify window with abret
        assert not ((self.abret == False) and (self.includefac == True)) #cannot include factors unless calculating abnormal returns
        assert not ((self.abret == False) and (self.includecoef == True)) #cannot include factor coefs unless calculating abnormal returns
        assert not ((self.abret != False) and ('RET' not in self.get)) #can't calculate abnormal returns without getting returns
        assert not (self.cumret and ('RET' not in self.get)) #can't cumulate returns without getting returns
        assert not (self.cumret and (self.time == None)) #can't cumulate over a single period
        assert not ((self.drop_first == True) and (self.time == None)) #can't drop first shifted time period if there are none
        assert not ((self.drop_first == True) and (len(self.time) == 1)) #can't drop first shifted time period if there's only one
        assert not ((self.drop_first == True) and (self.cumret == False)) #no reason to drop first shifted time period if we're not cumulating


        #Check to make sure company identifier is valid
        assert self.coid in ('GVKEY','TICKER', 'PERMNO', 'PERMCO', 'NCUSIP', 'CUSIP6')
        if self.coid != 'PERMNO':
            self._log('Company ID is not PERMNO. Getting PERMNO.')
            if self.coid == 'GVKEY':
                self.df = get_gvkey_or_permno(self.df, self.date, get='PERMNO',
                                              other_byvars=self.other_byvars) #grabs permno from ccmxpf_linktable
                self._log('Pulled PERMNO from ccmxpf_linktable.')
            else: #all others go through dsenames
                self.df = merge_dsenames(self.df, on=self.coid, date=self.date,
                                        other_byvars=self.other_byvars) #grabs permno
                self._log('Pulled PERMNO from dsenames.')

        #Check to make sure no columns currently in the dataframe have the same name as columns
        #we will be adding
        for col in self.get:
            if col in self.df.columns:
                self.df.rename(columns={col: col + '_old'}, inplace=True)

        #Ensure other_byvars is a list
        if isinstance(self.other_byvars, str):
            self.other_byvars = [self.other_byvars]
        assert isinstance(self.other_byvars, (list, type(None)))
    
    def _load_crsp(self):
        if self.freq.lower() == 'm':
            if not self.loaded_m:
                self.__load_crsp()
            self.loaded_m = True
        if self.freq.lower() == 'd':
            if not self.loaded_d:
                self.__load_crsp()
            self.loaded_d = True
        
    def __load_crsp(self):
        #Check frequency
        if self.freq.lower() == 'm':
            filename = 'msf'
        elif self.freq.lower() == 'd':
            filename = 'dsf'
        else: raise ValueError('use m or d for frequency')
        if self.debug: filename += '_test' #debug datasets only have permnos 10516, 10517

        #Load in CRSP file
        self._log('Loading CRSP dataframe...')
        filepath = os.path.join(self.crsp_dir, filename + '.sas7bdat')
        self.crsp_dfs[self.freq.lower()] = load_sas(filepath)
        self._log('Loaded.')

        #Change date to datetime format
        self._log('Converting SAS date to Pandas format.')
        self.crsp_dfs[self.freq.lower()]['DATE'] = convert_sas_date_to_pandas_date(
                                                    self.crsp_dfs[self.freq.lower()]['DATE'])
        self._log('Converted.')
    
    def _log(self, message):
        if message != '\n':
            time = datetime.datetime.now().replace(microsecond=0)
            message = str(time) + ': ' + message
        sys.stdout.write(message + '\n')
        sys.stdout.flush()
    
    def _merge_crsp(self, df, date):
        self._log('Merging CRSP to dataframe.')
        df = df.copy()

        #If we are using the monthly file, we need to merge on month and year
        if self.freq.lower() == 'm':
            self.crsp_df = self.crsp_dfs['m']
            self.crsp_df = year_month_from_date(self.crsp_df, date='DATE')
            orig_df_for_merge = year_month_from_date(df, date=date)

            #Now perform merge
            merged = orig_df_for_merge.merge(self.crsp_df[['Month','Year','PERMNO'] + self.get],
                                   how='left', on=['PERMNO','Month','Year'])
            merged.drop(['Month','Year'], axis=1, inplace=True)
            
        if self.freq.lower() == 'd':
            self.crsp_df = self.crsp_dfs['d']
            merged = df.merge(self.crsp_df[['DATE','PERMNO'] + self.get],
                              how='left', right_on=['PERMNO','DATE'],
                              left_on=['PERMNO', date])
            merged.drop('DATE', axis=1, inplace=True)
            
        self._log('Completed merge.')

        #Temp
        return merged

    
def get_ff_factors(df, fulldatevar=None, year_month=None, freq='m',
                   subset=None, ff_dir=r'C:\Users\derobertisna.UFAD\Desktop\Data\FF'):
    """
    Pulls Fama-French factors and merges them to dataset
    
    df: Input dataframe
    fulldatevar: String name of date variable to merge on. Specify this OR year and month variable. Must use this
                 and not year_month if pulling daily factors. If merging with monthly factors, will create month
                 and year variables in the output dataset. Warning: Will overwrite any variables called Month
                 and Year in the input data.
    year_month: Two element list of ['yearvar','monthvar']. Specify this OR full date variable.
    freq: 'm' for monthly factors, 'd' for daily
    subset: str or list, names of ff factors to pull. Can specify any of 'mktrf', 'smb', 'hml', 'umd'
    ff_dir: folder containing FF data
    """
   
    #Make sure inputs are correct
    assert isinstance(df, pd.DataFrame)
    assert freq in ('d','m')
    assert isinstance(ff_dir, str)
    assert not (fulldatevar == None and year_month == None)
    assert not (fulldatevar == None and freq == 'd')
    
    df = df.copy()
    
    if not subset:
        subset = ['mktrf', 'smb', 'hml', 'umd']
    if isinstance(subset, str):
        subset = [subset]
    assert isinstance(subset, list)
    for item in subset:
        assert item in ['mktrf', 'smb', 'hml', 'umd']
        
    subset = subset.copy() #don't modify original beyond converting to list
    
    if freq == 'm': 
        ff_name = 'ff_fac_month.sas7bdat'
        drop = False
        if year_month != None:
            left_datevars = year_month
            df_for_merge = df
        else: #fulldatevar specified
            df_for_merge = year_month_from_date(df, date=fulldatevar)
            left_datevars = ['Year','Month']
            drop = True
        right_datevars = ['year','month']
    else: 
        drop = False
        df_for_merge = df
        ff_name = 'ff_fac_daily.sas7bdat'
        left_datevars = fulldatevar
        right_datevars = ['date']
        
    subset += right_datevars + ['rf'] #need to pull date variables and risk free rate as well
        
    path = os.path.join(ff_dir, ff_name)
    ffdf = load_sas(path)
    ffdf['date'] = convert_sas_date_to_pandas_date(ffdf['date']) #convert to date object
    
    merged = df_for_merge.merge(ffdf[subset], how='left', left_on=left_datevars, right_on=right_datevars)
    merged.drop(right_datevars, axis=1, inplace=True)
    
    if drop:
        merged.drop(left_datevars, axis=1, inplace=True)
    
    return merged

def get_abret(df, byvars, fulldatevar='Date', year_month=None, freq='m', abret_fac=4, retvar='RET',
              includecoef=False, includefac=False):
    """
    Takes a dataframe containing a column of returns, dates, and at least one by variable and calculates
    abnormal returns for the model of choice. Returns a dataframe with the abnormal returns merged.
    
    Required inputs:
    df: pandas dataframe
    byvars: str or list of strs, column names of columns on which to form by groups
    fulldatevar: str, name of column containing date variable. If provided, don't provide year_month.
    year_month: list of strs, columns names of year and month variables, e.g. ['Year','Month']. Must
                set fulldatevar to None if year_month is provided.
    freq: 'm' or 'd', 'm' for monthly returns, 'd' for daily returns
    abret_fac: int (1, 3, 4), abnormal return model
    retvar: str, name of return variable
    includecoef: bool, set to True to get factor loadings
    includefac: bool, set to True to get factors and risk free rate
    """
    assert abret_fac in (1, 3, 4)
    factors = ['mktrf']
    if abret_fac >= 3:
        factors += ['smb','hml']
    if abret_fac == 4:
        factors += ['umd']
    
    out = get_ff_factors(df, fulldatevar=fulldatevar, freq=freq, subset=factors, year_month=year_month)
    out = factor_reg_by(out, byvars, fac=abret_fac, retvar=retvar)
    
    if not includefac:
        out.drop(factors + ['rf'], axis=1, inplace=True)
    if not includecoef:
        out.drop(['const'] + ['coef_' + fac for fac in factors], axis=1, inplace=True)
        
    return out

def load_and_merge_compustat(df, get=['sale'], freq='a', gvkeyvar='gvkey', datevar='Date', debug=False,
                             comp_dir=r'C:\Users\derobertisna.UFAD\Desktop\Data\Compustat'):
    """
    Convenience function for both loading and merging compustat to existing dataframe.
    
    Required inputs:
    df: Pandas dataframe containing a date variable and gvkey
    
    Optional inputs:
    get: List of strings, variable names to pull from compustat, not including those needed for
         the merge. Do not add the 'q' or 'y' for quarterly variables, this is done automatically.
         'y' variables will be converted to quarterly values by looking at changes.
    freq: string, 'a' or 'q' for annual or quarterly compustat (funda, fundq)
    gvkeyvar: string, name of variable containing gvkeys
    datevar: string, name of date variable in df on which to merge. Will pull the newest data
             that is before or on this date.
    debug: bool, True to restrict to only gvkeys (001076, 001722)
    comp_dir: string, directory containing compustat files
    """
    convert_gvkey(df, gvkeyvar)
    comp = load_compustat(freq, get=get, debug=debug, comp_dir=comp_dir)
    return merge_compustat(df, comp, datevar=datevar).rename(columns={'gvkey':gvkeyvar})

Overwriting data.py


# Compustat

In [23]:
%%writefile compustat.py

import os
import pandas as pd
from numpy import nan, float64, issubdtype, number

from .ext_pandas import convert_sas_date_to_pandas_date, load_sas, left_merge_latest

def compustat_keep_mask(df):
    return (df['indfmt'] == 'INDL') & (df['datafmt'] == 'STD') & \
           (df['popsrc'] == 'D')    & (df['consol'] == 'C')
    
def add_q_or_y(get, freq, cols):
    """
    Takes a list of get vars and adds q or y to work with quarterly file
    """
    if freq in ('q', 'quarterly'):
        return q_or_y(get, cols)
    else:
        return get
    
def q_or_y(get, cols):
    """
    Checks compustat cols to see which extension ('q', 'y', none) is appropriate and adds it.
    For use with quarterly data.
    """
    out_list = []
    for g in get:
        if g in ('fqtr','cusip','fyr','tic','conm'):
            out_list.append(g)
            continue
        
        q = g + 'q'
        y = g + 'y'
        if q in cols:
            name = q
        elif y in cols:
            name = y
        else:
            raise ValueError('variable {} does not have a quarterly counterpart'.format(g))
        out_list.append(name)
    return out_list

def create_q_from_y(df, var_y):
    """
    Single variable conversion
    Creates a compustat quarterly "q" variable from a compustat quarterly "y" variable. "q" variables
    are for just what happened in the quarter, while "y" variables are year to date. 
    
    Note: inplace
    """
    var = var_y[:-1] #gets name of variable without y
    df[var_y + '_lag'] = df[var_y].shift(1)
    df.loc[(df['fqtr'] > 1) & (df['gvkey'] == df['gvkey_lag']), var + 'q'] = \
                df[var_y] - df[var_y + '_lag']
    df.loc[df['fqtr'] == 1, var + 'q'] = df[var_y]
    
#     var = var_y[:-1] #gets name of variable without y
#     df[var_y + '_lag'] = df[var_y].shift(1)
#     df.loc[df['fqtr'] > 1, var + 'q'] = df[var_y] - df[var_y + '_lag']
#     df.loc[df['fqtr'] == 1, var + 'q'] = df[var_y]
    
def create_qs_from_ys(df, get, freq):
    """
    Dataframe conversion
    Creates compustat quarterly "q" variablse from a compustat quarterly "y" variables. "q" variables
    are for just what happened in the quarter, while "y" variables are year to date. 
    
    Note: will make edits to prior df even though returns a new df
    """
    if freq in ('q', 'quarterly'):
        y_get = [g for g in get if g.endswith('y')]
        df['gvkey_lag'] = df['gvkey'].shift(1)
        [create_q_from_y(df, g) for g in y_get]
        return df.drop(y_get + ['fqtr', 'gvkey_lag'] + [c + '_lag' for c in y_get], axis=1)
    else:
        return df
    
def check_freq(freq):
    freq = freq.lower()
    assert freq in ('a','q','annual','quarterly')
    return freq

def freq_to_name(freq, debug):
    if freq in ('a','annual'):
        name = 'funda'
    elif freq in ('q', 'quarterly'):
        name = 'fundq'
    if debug:
        name += '_test'
    name += '.sas7bdat'
    return name
    
def keep_relevant_data_compustat(df, get=['sale'], freq='a'):
    get = add_q_or_y(get, freq, df.columns) #adds 'q' or 'y' to getvars if freq='q'
    mask = compustat_keep_mask(df)
    keepvars = ['gvkey','datadate']
    if freq in ('q', 'quarterly'):
        keepvars += ['fqtr'] #need for converting 'y' variables to 'q' variables
    keepvars += get
    comp_y = df.loc[mask, keepvars].drop_duplicates(
        subset=['gvkey', 'datadate']).reset_index(drop=True)
    #comp_y includes 'y' vars, need to convert to 'q' vars 
    return create_qs_from_ys(comp_y, get, freq)

def convert_date_compustat(df):
    df['datadate'] = convert_sas_date_to_pandas_date(df['datadate'])
    
def load_compustat(freq, get=['sale'], debug=False, comp_dir=r'C:\Users\derobertisna.UFAD\Desktop\Data\Compustat'):
    freq = check_freq(freq)
    name = freq_to_name(freq, debug)
    path = os.path.join(comp_dir, name)
    comp = load_sas(path, dtype={'gvkey':str})
    comp = keep_relevant_data_compustat(comp, get=get, freq=freq)
    convert_date_compustat(comp)
    return comp

def merge_compustat(df, compdf, datevar='Date'):
    return left_merge_latest(df, compdf, 'gvkey',
                            left_datevar=datevar, right_datevar='datadate')

# def merge_compustat(df, compdf, datevar='Date'):
#     many = df.merge(compdf, on='gvkey', how='left')
#     lt = many.loc[many[datevar] >= many['datadate']] #left with datadates less than date

#     #find rows within groups which have the maximum datadate (soonest before given date)
#     data_rows = lt.groupby(['gvkey',datevar], as_index=False)['datadate'].max() \
#         .merge(lt, on=['gvkey', datevar, 'datadate'], how='left')
    
#     return df.merge(data_rows, on=['gvkey',datevar], how='left')

def convert_numeric_gvkey_to_string(gvkey):
    """
    Converts a single numeric gvkey to string
    """
    str_gvkey = str(int(gvkey))
    num_zeroes = 6 - len(str_gvkey)
    return '0' * num_zeroes + str_gvkey

def convert_gvkey_col(gvkey):
    """
    Converts a column of numeric gvkeys to string
    """
    if pd.isnull(gvkey): return nan
    else:
        return convert_numeric_gvkey_to_string(gvkey)
    
def convert_gvkey(df, gvkeyvar):
    """
    Renames gvkeyvar to 'gvkey' and converts to string if necessary
    
    Note: this is inplace
    """
    if gvkeyvar != 'gvkey':
        df.rename(columns={gvkeyvar: 'gvkey'}, inplace=True)
    if issubdtype(df['gvkey'].dtype, number):
        df['gvkey'] = df['gvkey'].apply(convert_gvkey_col)


Overwriting compustat.py


# Logging

In [24]:
%%writefile ext_logging.py

import datetime, os, sys
import logging, functools
import inspect
import timeit

from .ext_time import time_elapsed
from .decorators import apply_decorator_to_all_functions_in_module

def apply_logging_to_all_functions_in_module(module):
    """
    To be used after creating a logger with dero.logging.create_logger(), and after importing
    a module. On subsequent calls to any functions from that module, they will be logged using
    the log_with decorator. 
    
    NOTE: Be careful not to use this on any module containing a function to be called many times.
    For such modules, it is better to use the log_with decorator directly excluding those functions.
    
    Usage:
    import module
    import dero
    
    logger = dero.logging.create_logger()

    dero.logging.apply_logging_to_all_functions_in_module(module)
    
    module.whatever_function() #logs correctly
    
    """
    name = _get_all_prior_frames()
    name += '.' + module.__name__
    module.logger = logging.getLogger(name)
    module.log = log_with(module.logger)
    apply_decorator_to_all_functions_in_module(module, module.log)

def create_logger(name='main'):
    """
    Creates a logger in the __main__ namespace. Sets three handlers, two to file and one to stdout.
    All output goes to the .debug file, info and higher goes to the .log file, and error and higher
    goes to stdout.
    
    Pass a name to name log files.
    
    Usage:
    Imagine a project with three files, main.py, bar.py, and baz.py. We want to use the 
    create_logger() function in the main namespace (file being run), and get_logger() in
    the imported files.
    
    Normal logs:
    Then log entries may be created with logger.debug(), logger.info(), logger.warning(), logger.error(),
    and logger.critical(). 
    
    Exceptions:
    Log caught exceptions with logger.exception('Custom message'), this will include the traceback
    
    Entering and exiting functions:
    Use @dero.logging.log_with(logger) decorator, logs when entering and exiting function as well as
    passed args and kwargs and return values. Logs enter and exit at the info level and parameters and
    return values at the debug level.
    
    Example usage:
    main.py:
    import dero
    
    logger = dero.logging.create_logger()

    logger.info('Starting main')
    bar.barf()
    
    bar.py:
    import dero
    import baz
    
    logger = dero.logging.get_logger()
    
    def barf():
        logger.info('some info about barf')
        baz.baz()
        
    baz.py:
    import dero
    
    logger = dero.logging.get_logger()
    
    def baz():
        logger.info('some info about baz')
        
    Running main.py will output:
    2016-08-08 15:09:17,109 - __main__ - INFO - Starting main
    2016-08-08 15:09:17,111 - __main__.bar - INFO - some info about barf
    2016-08-08 15:09:17,111 - __main__.bar.baz - INFO - some info about baz

    """
    #Clear Jupyter notebook logger (this is code that only needs to be run in jupyter notebook)
    logger = logging.getLogger()
    logger.handlers = []

    #Create logger
    logger = logging.getLogger('__main__')
    logger.setLevel(logging.DEBUG)

    handlers = [] #container for handlers
    
    #Make log dir
    if not os.path.exists('Logs'): os.makedirs('Logs')

    #Create debug logfile which logs everything
    creation_time = str(datetime.datetime.now().replace(microsecond=0)).replace(':','.')
    debug_handler = logging.FileHandler(r'Logs\{} {}.debug'.format(creation_time, name))
    debug_handler.setLevel(logging.DEBUG)
    handlers.append(debug_handler)

    #Create standard logfile which logs process (info and up)
    info_handler = logging.FileHandler(r'Logs\{} {}.log'.format(creation_time, name))
    info_handler.setLevel(logging.INFO)
    handlers.append(info_handler)

    #Now log errors to standard output
    error_handler = logging.StreamHandler(sys.stdout)
    error_handler.setLevel(logging.ERROR)
    handlers.append(error_handler)

    formatter = logging.Formatter('%(asctime)ls - %(name)s - %(levelname)s - %(message)s')

    for handler in handlers:
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        
    return logger

def get_logger():
    """
    To be used in an imported file. See create_logger() for usage.
    """
    name = _get_all_prior_frames()
    return logging.getLogger(name)

def _get_all_prior_frames():
    """
    Gets the calling stack formatted as a string seperated by periods, e.g.:
    __main__.bar.baz
    """
    frame = inspect.currentframe()
    out = [] #container for output
    while True:
        frame = frame.f_back
        name = _filter_frame(frame)
        if frame is not None:
            if name is not False: #if False, is a name we don't need to record, should just continue
                out = [name] + out
                if name == '__main__': #once we get to __main__, we're done (ignore IPython stuff)
                    return '.'.join(out)
        else: #if frame is none, we're done (no more frames)
            return '.'.join(out)

def _filter_frame(frame):
    """
    Checks if this frame is something meaningful and takes the appropriate action
    
    Returns the name if valid name, returns False if invalid name, returns None if frame is None
    """
    try: name = frame.f_globals['__name__']
    except AttributeError: #frame is None
        return None
    if name in ('importlib._bootstrap','importlib._bootstrap_external', __name__):
        return False
    return name

def get_func_signature(func):
    code_list = inspect.getsourcelines(func)[0]
    code_str = ' '.join([c.strip() for c in code_list])
    return code_str[code_str.find('def') + 4:code_str.find(':')]

class log_with(object):
    '''Logging decorator that allows you to log with a
    specific logger.
    
    By default, logs entering and exiting function as well as arguments passed at the info level.
    
    Usage:
    import logging
    import dero
    
    logging.basicConfig()
    log = logging.getLogger('__name__') #can use custom name but using module name comes with benefits
    log.setLevel(logging.DEBUG)

    @dero.logging.log_with(log)
    def test_func(a, b, c=5):
        return a + b
    '''
    # Customize these messages
    ENTRY_MESSAGE = 'Entering {}'
    args_message = 'Passed Args: \n{}, Kwargs: {}'
    result_message = '{} Result: \n{}'
    time_message = '{} took {}'
    EXIT_MESSAGE = 'Exiting {}'

    def __init__(self, logger=None, timer=True):
        self.logger = logger
        self.timer = timer

    def __call__(self, func):
        '''Returns a wrapper that wraps func.
The wrapper will log the entry and exit points of the function
with logging.INFO level.
'''
        # set logger if it was not set earlier
        if not self.logger:
            logging.basicConfig()
            self.logger = logging.getLogger(func.__module__)

        @functools.wraps(func)
        def wrapper(*args, **kwds):
            if self.timer:
                start_time = timeit.default_timer()
            
            
            self.logger.info(self.ENTRY_MESSAGE.format(get_func_signature(func)))  # logging level .info(). Set to .debug() if you want to
            self.logger.debug(self.args_message.format(args, kwds))
            f_result = func(*args, **kwds)
            self.logger.debug(self.result_message.format(func.__name__, f_result))
            time_elapsed_str = time_elapsed(timeit.default_timer() - start_time)
            self.logger.debug(self.time_message.format(func.__name__, time_elapsed_str))
            self.logger.info(self.EXIT_MESSAGE.format(func.__name__))   # logging level .info(). Set to .debug() if you want to
            return f_result
        return wrapper

class Logger:
    
    def __init__(self, log_dir):
        self.log_dir = log_dir
        
        self.log_list = []
        self.create_log_file()

    def log(self, message, error=False, neverprint=False):
        if error:
            message = 'ERROR: ' + message
        if message != '\n':
            time = datetime.datetime.now().replace(microsecond=0)
            message = str(time) + ': ' + message
        if self.debug and not neverprint:
            sys.stdout.write(message + '\n')
            sys.stdout.flush() #forces output now
        try:
            with open(self.log_path, 'a') as f:
                [f.write(item) for item in self.log_list] #log anything saved in memory that couldn't be written before
                f.write(message)
                f.write('\n')
            self.log_list = []
        except PermissionError: #if someone happened to write to the file at the same time
            self.log_list.append(message) #save it to log later
            self.log_list.append('\n')

    def create_log_file(self):
        name = 'log_' + str(datetime.datetime.now().replace(microsecond=0)).replace(':','.') + '.txt'
        if not os.path.exists(self.log_dir): os.makedirs(self.log_dir)
        self.log_path = os.path.join(self.log_dir, name)

        if not os.path.exists(self.log_path):
            with open(self.log_path, 'w') as f:
                f.write('\n')


        

Overwriting ext_logging.py


# Logtimer

In [25]:
%%writefile logtimer.py

import re
import pandas as pd
import numpy as np
import dero

from .ext_pandas import groupby_merge

def load_log_and_produce_timing_dfs(filepath):
    """
    Loads a dero log from filepath and returns a tuple of two pandas dataframes, where the
    first is a listing of each function in order and the amount of time it took,
    and the second is a summary df which sums and averages times by functions (across
    multiple calls)
    """
    with open(filepath, 'r') as f:
        log_list = f.readlines()

    df = parse_logs_for_timing(log_list)
    summ_df = summary_timing_df(df)
    
    return df, summ_df

def compare_log_timing(filepaths, substrings):
    r"""
    Loads multiple logs and produces a summary df with how total and average times per functions on the
    different runs. Use substrings to identify which sample is which.
    
    Required inputs:
    filepaths: list of strs, locations of logs
    substrings: list of strs, must be same length as filepaths list, identifies each sample
    
    Usage:
    filepaths = [
    r'C:\Users\derobertisna.UFAD\Dropbox\UF\Investor Attention\Python\2016-08-08 23.37.24 ia.debug', #20 subset
    r'C:\Users\derobertisna.UFAD\Dropbox\UF\Investor Attention\Python\2016-08-09 00.02.53 ia.debug' #200 subset
    ]
    substrings = ['20', '200']

    dero.logtimer.compare_log_timing(filepaths, substrings)
    
    """
    df_tups = [load_log_and_produce_timing_dfs(fp) for fp in filepaths]
    summ_dfs = [tup[1] for tup in df_tups]
    
    for i, summ_df in enumerate(summ_dfs):
        suff = substrings[i] #get suffix
        summ_df = summ_df.rename(columns={'time_sum':'time_sum' + suff, 'time_mean':'time_mean' + suff})
        if i == 0:
            alldf = summ_df.copy()
            continue
        alldf = alldf.merge(summ_df, on='function', how='outer')
    
    diff_col = 'diff_' + substrings[0] + '_' + substrings[-1]
    alldf[diff_col] = alldf['time_sum' + substrings[-1]] - alldf['time_sum' + substrings[0]]
    return alldf.sort_values(diff_col, ascending=False)

def _parse_log_entry(l):
    """
    Returns an re match object with the contents:
    
    First group: timestamp
    2: module name
    3: logging level
    4: message
    """
    pattern = re.compile(r'(\d*-\d*-\d* \d*:\d*:\d*,\d*) - ([\w.]+) - (\w*) - (.+)')
    return pattern.match(l)

def _parse_timing_entry(l):
    """
    Returns a tuple of full function name, time elapsed strings 
    """
    match = _parse_log_entry(l)
    if not match: return None
    timing_pattern = re.compile(r'([\w.]+) took (.+)[.]')
    function_time = timing_pattern.match(match.group(4))
    if not function_time: return None
    full_function_str = match.group(2) + '.' +  function_time.group(1)
    return full_function_str, function_time.group(2)

def _parse_logs_for_timing(log_list):
    """
    Returns a list of tuples of full function name, time elapsed strings
    """
    return list(filter(lambda x: x, [_parse_timing_entry(l) for l in log_list]))

def parse_logs_for_timing(log_list):
    """
    Returns a tuple of two pandas dataframes, where the first is
    full function name, time elapsed
    and the 
    """
    df = pd.DataFrame(_parse_logs_for_timing(log_list), columns=['function','time'])
    df['time'] = pd.to_timedelta(df['time']).apply(lambda x: x.total_seconds())
    return df.sort_values('time', ascending=False).reset_index().rename(columns={'index':'orig order'})

def summary_timing_df(parsed_df):
    df = groupby_merge(parsed_df, 'function', 'sum', subset='time')
    df = groupby_merge(df, 'function', 'mean', subset='time')
    return df.drop(['time','orig order'], axis=1).drop_duplicates(
        ).sort_values('time_sum', ascending=False)



Overwriting logtimer.py


# Matplotlib

In [26]:
%%writefile ext_matplotlib.py

def setAxLinesBW(ax):
    """
    Take each Line2D in the axes, ax, and convert the line style to be 
    suitable for black and white viewing.
    """
    MARKERSIZE = 3

    COLORMAP = {
        'b': {'marker': None, 'dash': (None,None)},
        'g': {'marker': None, 'dash': [5,5]},
        'r': {'marker': None, 'dash': [5,3,1,3]},
        'c': {'marker': None, 'dash': [1,3]},
        'm': {'marker': None, 'dash': [5,2,5,2,5,10]},
        'y': {'marker': None, 'dash': [5,3,1,2,1,10]},
        'k': {'marker': 'o', 'dash': (None,None)} #[1,2,1,10]}
        }


    lines_to_adjust = ax.get_lines()
    try:
        lines_to_adjust += ax.get_legend().get_lines()
    except AttributeError:
        pass

    for line in lines_to_adjust:
        origColor = line.get_color()
        line.set_color('black')
        line.set_dashes(COLORMAP[origColor]['dash'])
        line.set_marker(COLORMAP[origColor]['marker'])
        line.set_markersize(MARKERSIZE)

def set_fig_bw(fig):
    """
    Take each axes in the figure, and for each line in the axes, make the
    line viewable in black and white.
    """
    for ax in fig.get_axes():
        setAxLinesBW(ax)

Overwriting ext_matplotlib.py


# PDF

In [27]:
%%writefile pdf.py

import os
from pdfrw import PdfReader, PdfWriter

def strip_pages_pdf(indir, infile, outdir=None, outfile=None, numpages=1, keep=False):
    '''
    Deletes the first pages from a PDF. Omit outfile name to replace. Default is one page.
    If option keep is specified, keeps first pages of PDF, dropping rest.
    '''
    if outfile is None:
        outfile = infile
        
    if outdir is None:
        outdir = indir

    output = PdfWriter()
    inpath = os.path.join(indir,infile)
    outpath = os.path.join(outdir,outfile)
    
    for i, page in enumerate(PdfReader(inpath).pages):
        if not keep:
            if i > (numpages - 1):
                output.addpage(page)
        if keep:
            if i <= (numpages - 1):
                output.addpage(page)

    output.write(outpath)

Overwriting pdf.py


# Future (not written to file)

# Setup for PyPi

In [28]:
os.chdir(r'C:\Users\derobertisna.UFAD')

In [29]:
%%writefile .pypirc

[distutils]
index-servers=pypi

[pypi]
repository = https://pypi.python.org/pypi
username = whoopnip

Overwriting .pypirc


Run below to upload to PyPi. Use the first file if it's the first upload, and the second for recurring uploads.

In [30]:
os.chdir(orig_path)

In [31]:
first_time_upload_str = '''
python setup.py sdist bdist_wheel
twine register dist\Dero-{0}*
twine upload dist\Dero-{0}*
pause
'''.format(version)

with open('first_time_upload.bat', 'w') as f:
    f.write(first_time_upload_str)

In [32]:
upload_str = '''
python setup.py sdist bdist_wheel
twine upload dist\Dero-{}*
pause
'''.format(version)
with open('upload.bat', 'w') as f:
    f.write(upload_str)

In [33]:
if upload:
    os.system('start cmd /C upload.bat')

In [34]:
dero_path = os.path.join(orig_path, 'dero')
os.chdir(dero_path)